In [ ]:
from datetime import timedelta
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob

# LOAD DATA FROM COW DATABASE

Requires the following data to create dfKok.csv:
- Kok_Calving
- Kok_HerdEntryExit
- Kok_CowMilkSampling
- Kok_Lineage
- Kok_Reproduction

In [ ]:
#
#
# LOAD COW DATABASE CALVING DATA
calving = pd.read_csv("Kok_Calving240820.csv", delimiter=';', low_memory=False)
col_keep = ["ActiveHerdNumber", "BirthID", "CalvingDate", "CalvingNumber"]
calving = calving[col_keep]
calving.rename(columns={'BirthID': 'SE_Number', "ActiveHerdNumber": "FarmName_Pseudo", "CalvingDate": "CalvingDateKok",
                        "CalvingNumber": "LactationNumberKok"}, inplace=True)
calving = calving.sort_values(by=["SE_Number", "CalvingDateKok"])
calving = calving.drop_duplicates(subset=["SE_Number", "CalvingDateKok"])
# calving["upper_limit"] = calving.groupby(["SE_Number"])["CalvingDateKok"].shift(-1)
calving.to_csv("calving_kok.csv", index=False)

calving2 = calving.groupby(["FarmName_Pseudo"])["LactationNumberKok"].count().reset_index()
calving2.rename(columns={'LactationNumberKok': 'CountLact'}, inplace=True)
print(f"No. of lactation records in cow database in different herds: \n", calving2.to_string(index=False))
calving2 = calving.drop_duplicates(subset=["SE_Number", "CalvingDateKok"])
print(f"No. of lactation records in cow database: {calving2.shape}")  # 23,688

calving2 = calving.groupby(["FarmName_Pseudo"])["SE_Number"].count().reset_index()
calving2.rename(columns={'SE_Number': 'CountCows'}, inplace=True)
print(f"No. of cows with calving data in cow database in different herds: \n", calving2.to_string(index=False))
calving2 = calving.drop_duplicates(subset=["SE_Number"])
print(f"No. of cows with calving data in cow database: {calving2.shape}")  # 9,168

In [ ]:
#
#
# ADD ENRTY AND CULLING DATA FROM COW DATABASE
cull = pd.read_csv("Kok_HerdEntryExit240820.csv", delimiter=';', low_memory=False)
col_keep = ["BirthID", "ActiveHerdNumber", "EntryDate", "ExitDate", "ExitReason_PrimaryReason",
            "ExitReason_SecondaryReason1","ExitReason_SecondaryReason2"]
cull = cull[col_keep]
cull.rename(columns={'BirthID': 'SE_Number', "ActiveHerdNumber": "FarmName_Pseudo", "EntryDate": "EntryDateKok",
                     "ExitDate": "ExitDateKok", "ExitReason_PrimaryReason": "ExitReason_PrimaryReasonKok",
                     "ExitReason_SecondaryReason1": "ExitReason_SecondaryReason1Kok",
                     "ExitReason_SecondaryReason2": "ExitReason_SecondaryReason2Kok"}, inplace=True)
cull = cull.sort_values(by=["SE_Number", "ExitDateKok"])
print(f"Number of records in raw entry/culling file: {cull.shape}")  # 33,679
cull2 = cull.drop_duplicates(subset=["SE_Number", "ExitDateKok"])
print(f"NUmber of unique records in entry/culling file: {cull2.shape}")  # 33,668
# => 11 records with multiple culling reasons

# check how many cows enter/exit herds multiple times
cull3 = cull2.groupby(["SE_Number"])["ExitDateKok"].count().reset_index()
cull3.rename(columns={'ExitDateKok': 'CountExits'}, inplace=True)

frequency_table = cull3['CountExits'].value_counts()
print(f"No. of cows with multiple exit records in cow database:")
print(frequency_table)

# Assume last record within cow as culling date
cull2 = cull2.groupby('SE_Number').tail(1)
cull2.to_csv("cullingKok.csv", index=False)

# Merge
dfKok = calving.merge(cull2, on=["FarmName_Pseudo", "SE_Number"], how="left")

# Make upper_limit to sort dry off date later
dfKok = dfKok.sort_values(by=["SE_Number", "CalvingDateKok"]).reset_index(drop=True)
dfKok["upper_limit"] = dfKok.groupby(["SE_Number"])["CalvingDateKok"].shift(-1)

dfKok.to_csv("dfKok.csv", index=False)

In [ ]:
#
#
# ADD DRY OFF DATA FROM COW DATABASE
dry_offKok = pd.read_csv("Kok_Kok_CowMilkSampling240829.csv", delimiter=';', low_memory=False)
col_keep = ["BirthID", "ActiveHerdNumber", "SamplingDate", "VariousSystemInfo"]
dry_offKok = dry_offKok[col_keep]
dry_offKok.rename(columns={'BirthID': 'SE_Number', "ActiveHerdNumber": "FarmName_Pseudo",
                           "SamplingDate": "DryOffDateKok"}, inplace=True)
col_keep = ["FarmName_Pseudo", "SE_Number", "DryOffDateKok", "VariousSystemInfo"]
dry_offKok = dry_offKok[col_keep]

# Find dry off data and output last record for each cow (ie last time she's sampled for test day sampling)
# and by "kod: 02"
dry_offKok = dry_offKok.sort_values(by=["SE_Number", "DryOffDateKok"])
# Create a new column to track when the "VariousSystemInfo" changes, want the last one ie -1
dry_offKok['InfoChange'] = (dry_offKok.groupby('SE_Number')['VariousSystemInfo'].shift(-1) !=
                            dry_offKok['VariousSystemInfo'])
dry_offKok.to_csv("DryOffKok1.csv", index=False)
# Keep only the rows where there is a change in "VariousSystemInfo" AND "VariousSystemInfo" is 2 ie dry off
dry_offKok = dry_offKok[(dry_offKok["VariousSystemInfo"] == "kod: 02") & (dry_offKok["InfoChange"] == True)]
# Drop columns
dry_offKok = dry_offKok.drop(columns=['InfoChange', "VariousSystemInfo"])
dry_offKok.to_csv("DryOffKok.csv", index=False)

print(f"No. dry off records in cow database: {dry_offKok.shape}")  # 12,513
dry_offKok2 = dry_offKok.drop_duplicates(subset=["SE_Number"])
print(f"No. cows with dry off records in cow database: {dry_offKok2.shape}")  # 6,447

# Merge
dfKok = dfKok.merge(dry_offKok, on=["FarmName_Pseudo", "SE_Number"], how="left")
dfKok["CalvingDateKok"] = pd.to_datetime(dfKok["CalvingDateKok"])
dfKok["DryOffDateKok"] = pd.to_datetime(dfKok["DryOffDateKok"])
dfKok["upper_limit"] = pd.to_datetime(dfKok["upper_limit"])


def data(row):
    if row["CalvingDateKok"] < row["DryOffDateKok"] <= row["upper_limit"]:
        return 1
    else:
        return 0


dfKok["dryoff"] = dfKok.apply(data, axis=1)

# Keep all records with dry off dates fitted within lactation
df_ones = dfKok[dfKok['dryoff'] == 1]
# Keep last record where open and lacking dry off date
df_last_zero = dfKok[dfKok['dryoff'] == 0].groupby("SE_Number").tail(1)
# Concatenate dataframe and sort to maintain original order
df_combined = pd.concat([df_ones, df_last_zero])
df_combined = df_combined.sort_values(by=["SE_Number", "CalvingDateKok"]).reset_index(drop=True)
df_combined.to_csv("dfKok.csv", index=False)

# Put ExitDateKok as upper_limit if upper_limit is missing from calving date (mostly last lactation)
df_combined.loc[df_combined["upper_limit"].isna() & df_combined["ExitDateKok"].notna(), "upper_limit"] = (
    df_combined)["ExitDateKok"]

# Get today's date for current lactation when missing upper_limit after adjusting using ExitDateKok
df_combined['TodayDate'] = pd.to_datetime('today').normalize()
df_combined.loc[df_combined["upper_limit"].isna() & df_combined["ExitDateKok"].isna(), "upper_limit"] = (
    df_combined)["TodayDate"]

df_combined.loc[df_combined["upper_limit"].isna()
                & (df_combined["DryOffDateKok"] < df_combined["CalvingDateKok"]), "upper_limit"] = (df_combined)["DryOffDateKok"]
df_combined.loc[df_combined["dryoff"] == 0, "DryOffDateKok"] = np.nan
df_combined.to_csv("dfKok.csv", index=False)

In [ ]:
#
#
# ADD BREED INFORMATION FROM COW DATABASE
df = pd.read_csv("Kok_Lineage240821.csv", delimiter=';', low_memory=False)
df.rename(columns={"BirthID": "SE_Number", "ActiveHerdNumber": "FarmName_Pseudo", "Father_Breed": "SireBreedKok",
                   "Mother_Breed": "DamBreedKok", "MothersFather_Breed": "MGSBreedKok"}, inplace=True)
col_keep = ["FarmName_Pseudo", "SE_Number", "SireBreedKok", "DamBreedKok", "MGSBreedKok"]
df = df[col_keep]

# Check for duplicates and sort
print(df.shape)  # 24,067
df = df.sort_values(by=["FarmName_Pseudo", "SE_Number"])
df2 = df.drop_duplicates(subset=["SE_Number", "SireBreedKok", "DamBreedKok", "MGSBreedKok"])
print(f"No. cows with breed data: {df2.shape}")  # 22,653 => 1414 duplicates, all okay
df3 = df.drop_duplicates(subset=["SE_Number"])
print(f"No. cows with different breed data recorded: {df3.shape}")  # 22,607 => 46 cows with conflicting breed

# The 46 cows with conflicting breed are a mess, e.g. LIM and SJB for the same sire, remove!
duplicate_records = df2[df2['SE_Number'].duplicated(keep=False)]
duplicate_records = duplicate_records.copy()
duplicate_records["DupBreed"] = 1
col_keep = ["SE_Number", "DupBreed"]
duplicate_records = duplicate_records[col_keep]
df_ras = df2.merge(duplicate_records, on=["SE_Number"], how="left")
df_ras = df_ras[df_ras['DupBreed'] != 1]
df_ras.to_csv("kok_ras.csv", index=False)

dfKok = pd.read_csv("dfKok.csv", low_memory=False)
dfKok = dfKok.merge(df_ras, on=["FarmName_Pseudo", "SE_Number"], how="left")
dfKok.to_csv("dfKok.csv", index=False)

In [ ]:
#
#
# ADD INSEMINATION FROM COW DATABASE
df13 = pd.read_csv("Kok_Reproduction240820.csv", delimiter=';', low_memory=False)

# Create a boolean mask where SireBull_SE_Number is NE 0
mask = df13["SireBull_SE_Number"] != 0
# Sum the mask to count the number of True values (i.e., the number of not 0s)
count_non_zeros = mask.sum()
print(f"Number of events with sire ID in cow database: {count_non_zeros}")  # 95,369

# Count occurrences of each unique value in the EventType column
value_counts = df13["EventType"].value_counts()
print(value_counts)
"""
EventType
Inseminering               47926
Dräktighetsundersökning    41625
Behandling                  2120
Embryoinlägg                2092
Betäckning                  1433
Fri bet                      173
"""
# Keep only data from insemination
df14 = df13[df13["EventType"] == "Inseminering"]

# Check for duplicates and sort
print(f"No. insemination records in raw file in cow database: {df14.shape}")  # 47,926, 27col
df14 = df14.sort_values(by=["BirthID", "EventDate"])
df15 = df14.drop_duplicates(subset=["BirthID", "EventDate"])
print(f"No. unique inseminations in cow database: {df15.shape}")  # 43,951, 27col

df15 = df15.copy()
df15.rename(columns={"ActiveHerdNumber": "FarmName_Pseudo", "BirthID": "SE_Number", "EventDate": "InseminationDateKok",
                     "SireBull_SE_Number": "SireBull_SE_NumberKok"}, inplace=True)
col_keep = ["FarmName_Pseudo", "SE_Number", "InseminationDateKok", "SireBull_SE_NumberKok"]
df15 = df15[col_keep]

"""
# Subset chosen cow
SE_Number = ["SE-064c0cec-1189"]
df15 = df15[df15["SE_Number"].isin(SE_Number)]
"""
"""
# Subset chosen cows
SE_Number = ["SE-064c0cec-1189", "SE-5c06d92d-3145", "SE-5c06d92d-3177", "SE-5b581702-1742",
             "SE-5b581702-1851", "SE-5c06d92d-2915", "SE-5b581702-2002", "SE-5c06d92d-2515"]
df15 = df15[df15["SE_Number"].isin(SE_Number)]
"""

# Merge with calving data
dfkok2 = pd.read_csv("dfKok.csv", low_memory=False)
dfins6 = pd.merge(dfkok2, df15, on=["FarmName_Pseudo", "SE_Number"])

# Filter df for relevant inseminations sorted to correct lactation
dfins6 = dfins6[(dfins6["InseminationDateKok"] >= dfins6["CalvingDateKok"]) & (dfins6["InseminationDateKok"] <= dfins6["upper_limit"])]
dfins6.to_csv("dfKok.csv", index=False)

In [ ]:
#
#
# ADD PREGNANCY CHECKS FROM COW DATABASE
# Make next_ins to sort pregnancy checks
df20 = pd.read_csv("dfKok.csv", low_memory=False)
# df20 = pd.DataFrame(df20, columns=["SE_Number", "LactationNumberKok", "InseminationDateKok", "upper_limit"])

df20['next_ins'] = df20.groupby(['SE_Number', 'LactationNumberKok'])['InseminationDateKok'].shift(-1)

# only keep next_ins where falls within range
dfins7 = df20[(df20["next_ins"] >= df20["InseminationDateKok"]) & (df20["next_ins"] <= df20["upper_limit"])]
col_keep = ["SE_Number", "LactationNumberKok", "InseminationDateKok", "next_ins", "upper_limit"]
dfins7 = dfins7[col_keep]
dfins7.to_csv("dfKok2.csv", index=False)

"""
#Subset chosen cow
SE_Number = ["SE-064c0cec-1189"]
dfins8 = dfins8[dfins8["SE_Number"].isin(SE_Number)]
#dfins8.to_csv("dataframe3.csv", index=False)
"""

# Load pregnancy check data, check for duplicates, sort
preg = pd.read_csv("Kok_Reproduction240820.csv", delimiter=';', low_memory=False)
# Keep only data from pregnancy checks
preg = preg[preg["EventType"] == "Dräktighetsundersökning"]

col_keep = ["BirthID", "EventDate", "PregnancyStatus"]
preg = preg[col_keep]
preg.rename(columns={"BirthID": "SE_Number", "EventDate": "PregnancyCheckDateKok",
                     "PregnancyStatus": "PregnancyStatusKok"}, inplace=True)

print(f"No. pregnancy checks in cow database: {preg.shape}")  # 41,625 events, 3col
preg = preg.drop_duplicates(subset=["SE_Number", "PregnancyCheckDateKok"])
print(f"No. unique pregnancy checks in cow database: {preg.shape}")    # 38,320 unique events, 3col
preg = preg.sort_values(by=["SE_Number", "PregnancyCheckDateKok"])

# Add to subset df
col_keep = ["SE_Number", "LactationNumberKok", "InseminationDateKok", "next_ins", "upper_limit"]
df20 = df20[col_keep]
df21 = df20.join(preg.set_index(["SE_Number"]), on=["SE_Number"])

# Ensure datetime conversion
df21['InseminationDateKok'] = pd.to_datetime(df21['InseminationDateKok'])
df21['PregnancyCheckDateKok'] = pd.to_datetime(df21['PregnancyCheckDateKok'])
df21['next_ins'] = pd.to_datetime(df21['next_ins'])
df21['upper_limit'] = pd.to_datetime(df21['upper_limit'])

# Initialize 'C' column with NaN
df21['C'] = np.nan


# Define the filtering function
def filter_pregcheck(row):
    if pd.isna(row["next_ins"]):
        if (row["PregnancyCheckDateKok"] >= row["InseminationDateKok"]) and (row["PregnancyCheckDateKok"]
                                                                             <= row["upper_limit"]):
            return "Yes"
        else:
            return "No"
    if pd.notna(row["next_ins"]):
        if (row["PregnancyCheckDateKok"] >= row["InseminationDateKok"]) and (row["PregnancyCheckDateKok"]
                                                                             <= row["next_ins"]):
            return "Yes"
        else:
            return "No"


# Apply the filter function to each row
df21['C'] = df21.apply(filter_pregcheck, axis=1)
dfins10 = df21[df21["C"] == "Yes"]
dfins10.to_csv("dfKok2.csv", index=False)

col_keep = ["SE_Number", "LactationNumberKok", "InseminationDateKok", "PregnancyCheckDateKok", "PregnancyStatusKok"]
dfins11 = dfins10[col_keep]

# Convert the 'InseminationDate' column from datetime64[ns] to object for merging
print(dfins11.dtypes)
dfins11 = dfins11.copy()
dfins11['InseminationDateKok'] = dfins11['InseminationDateKok'].astype(str)

# Add to master df
dfins12 = pd.read_csv("dfKok.csv")
print(dfins12.dtypes)
dfins13 = dfins12.merge(dfins11, on=["SE_Number", "LactationNumberKok", "InseminationDateKok"], how="left")

"""
#Subset chosen cow
SE_Number = ["SE-064c0cec-1189"]
dfins13 = dfins13[dfins13["SE_Number"].isin(SE_Number)]
"""
dfins13.to_csv("dfKok.csv", index=False)

# LOAD DELPRO DATA

Requires the following data to create dfDelPro.csv:
- Del_Calving
- Del_DryOff
- Del_Lactation
- Del_Cow
- Del_Insemination
- Del_PregnancyCheck

In [ ]:
#
#
# LOAD DELPRO CALVING DATA
calving = pd.read_csv("Del_Calving240823.csv", delimiter=';', low_memory=False)
col_keep = ["FarmName_Pseudo", "SE_Number", "CalvingDate"]
calving = calving[col_keep]
calving.rename(columns={"CalvingDate": "CalvingDateDelPro"}, inplace=True)
calving = calving.sort_values(by=["SE_Number", "CalvingDateDelPro"])
calving = calving.drop_duplicates(subset=["SE_Number", "CalvingDateDelPro"])
calving["upper_limit"] = calving.groupby(["SE_Number"])["CalvingDateDelPro"].shift(-1)
calving.to_csv("calving_delpro.csv", index=False)

calving2 = calving.groupby(["FarmName_Pseudo"])["CalvingDateDelPro"].count().reset_index()
calving2.rename(columns={'CalvingDateDelPro': 'CountLact'}, inplace=True)
print(f"No. of lactation records in DelPro in different herds: \n", calving2.to_string(index=False))
calving2 = calving.drop_duplicates(subset=["SE_Number", "CalvingDateDelPro"])
print(f"No. of lactation records in DelPro: {calving2.shape}")  # 10,163

calving2 = calving.groupby(["FarmName_Pseudo"])["SE_Number"].count().reset_index()
calving2.rename(columns={'SE_Number': 'CountCows'}, inplace=True)
print(f"No. of cows with calving data in DelPro in different herds: \n", calving2.to_string(index=False))
calving2 = calving.drop_duplicates(subset=["SE_Number"])
print(f"No. of cows with calving data in DelPro: {calving2.shape}")  # 5,401

# LOAD DELPRO CULLING DATA
culling = pd.read_csv("Del_Cow240823.csv", delimiter=';', low_memory=False)
col_keep = ["SE_Number", "BirthDate", "CullDecisionDate", "CullReason1", "CullReason2"]
culling = culling[col_keep]
culling = culling.sort_values(by=["SE_Number", "CullDecisionDate"])
print(f"No. records in raw culling file: {culling.shape}")  # 25,105
culling2 = culling.drop_duplicates(subset=["SE_Number", "CullDecisionDate"])
print(f"No. records in culling file: {culling.shape}")  # 25,105
culling2.to_csv("cull_delpro.csv", index=False)

# Merge
for_my_rec = calving.merge(culling2, on=["SE_Number"], how="left")
for_my_rec.loc[pd.isna(for_my_rec["upper_limit"]), "upper_limit"] = for_my_rec["CullDecisionDate"]
for_my_rec.to_csv("dfDelPro.csv", index=False)

In [ ]:
#
#
# LOAD DELPRO CULLING DATA
culling = pd.read_csv("Del_Cow240823.csv", delimiter=';', low_memory=False)
col_keep = ["SE_Number", "BirthDate", "CullDecisionDate", "CullReason1", "CullReason2"]
culling = culling[col_keep]
culling = culling.sort_values(by=["SE_Number", "CullDecisionDate"])
print(f"No. records in raw culling file: {culling.shape}")  # 25,105
culling2 = culling.drop_duplicates(subset=["SE_Number", "CullDecisionDate"])
print(f"No. records in culling file: {culling.shape}")  # 25,105
culling2.to_csv("cull_delpro.csv", index=False)

# Merge
for_my_rec = calving.merge(culling2, on=["SE_Number"], how="left")
for_my_rec.loc[pd.isna(for_my_rec["upper_limit"]), "upper_limit"] = for_my_rec["CullDecisionDate"]
for_my_rec.to_csv("dfDelPro.csv", index=False)

In [ ]:
#
#
# LOAD DELPRO DRY OFF DATA
dry_off = pd.read_csv("Del_DryOff240823.csv", delimiter=';', low_memory=False)
col_keep = ["SE_Number", "DryOffDate"]
dry_off = dry_off[col_keep]
dry_off = dry_off.sort_values(by=["SE_Number", "DryOffDate"])
dry_off = dry_off.drop_duplicates(subset=["SE_Number", "DryOffDate"])
print(f"No. records in dry off file: {dry_off.shape}")  # 5,305
dry_off.to_csv("dry_off_delpro.csv", index=False)

for_my_rec = for_my_rec.merge(dry_off, on=["SE_Number"], how="left")
for_my_rec["CalvingDateDelPro"] = pd.to_datetime(for_my_rec["CalvingDateDelPro"])
for_my_rec["DryOffDate"] = pd.to_datetime(for_my_rec["DryOffDate"])
for_my_rec["upper_limit"] = pd.to_datetime(for_my_rec["upper_limit"])


def data(row):
    if row["CalvingDateDelPro"] < row["DryOffDate"] <= row["upper_limit"]:
        return 1
    else:
        return 0


for_my_rec["dryoff"] = for_my_rec.apply(data, axis=1)

# Keep all records with dry off dates fitted within lactation
df_ones = for_my_rec[for_my_rec['dryoff'] == 1]
# Keep last record where open and lacking dry off date
df_last_zero = for_my_rec[for_my_rec['dryoff'] == 0].groupby("SE_Number").tail(1)
# Concatenate dataframe and sort to maintain original order
df_combined = pd.concat([df_ones, df_last_zero])
df_combined = df_combined.sort_values(by=["SE_Number", "CalvingDateDelPro"]).reset_index(drop=True)
df_combined.to_csv("dfDelPro.csv", index=False)

# Get today's date for current lactation missing upper_limit
df_combined['TodayDate'] = pd.to_datetime('today').normalize()
df_combined.loc[df_combined["upper_limit"].isna() & df_combined["CullDecisionDate"].isna(), "upper_limit"] = (
    df_combined)["TodayDate"]

df_combined.loc[df_combined["upper_limit"].isna()
                & (df_combined["DryOffDate"] < df_combined["CalvingDateDelPro"]), "upper_limit"] = df_combined[
    "DryOffDate"]
df_combined.loc[df_combined["dryoff"] == 0, "DryOffDate"] = np.nan
df_combined.to_csv("dfDelPro.csv", index=False)

In [ ]:
#
#
# LOAD DELPRO LACTATION NUMBER
lact = pd.read_csv("Del_Lactation240823.csv", delimiter=';', low_memory=False)
col_keep = ["SE_Number", "LactationInfoDate", "LactationNumber"]
lact = lact[col_keep]
lact = lact.sort_values(by=["SE_Number", "LactationInfoDate", "LactationNumber"])
lact = lact.drop_duplicates(subset=["SE_Number", "LactationNumber"])
lact.to_csv("lact.csv", index=False)

for_my_rec2 = lact.merge(df_combined, on=["SE_Number"], how="left")
for_my_rec2 = for_my_rec2[for_my_rec2["LactationInfoDate"] != "2022-05"]
for_my_rec2["LactationInfoDate"] = pd.to_datetime(for_my_rec2["LactationInfoDate"])


def data1(row):
    if row["CalvingDateDelPro"] <= row["LactationInfoDate"] <= row["upper_limit"]:
        return 1
    else:
        return 0


for_my_rec2["lact"] = for_my_rec2.apply(data1, axis=1)
for_my_rec2 = for_my_rec2[for_my_rec2['lact'] == 1]
for_my_rec2.to_csv("dfDelPro.csv", index=False)
print(f"No. of lactations in dataframe: {for_my_rec2.shape}")  # 10,361 lact - when DelPro data is used
for_my_rec3 = for_my_rec2.drop_duplicates("SE_Number")
print(f"No. of cows in dataframe: {for_my_rec3.shape}")  # 5,397

In [ ]:
#
#
# ADD DELPRO BREED INFORMATION
breed = pd.read_csv("Del_Cow240823.csv", delimiter=';', low_memory=False)
col_keep = ["SE_Number", "BreedName"]
breed = breed[col_keep]
breed = breed.sort_values(by=["SE_Number", "BreedName"])
breed = breed.drop_duplicates(subset=["SE_Number", "BreedName"])
breed.to_csv("breed.csv", index=False)

for_my_rec2 = for_my_rec2.merge(breed, on=["SE_Number"], how="left")  # - when Del_Calving is used
# for_my_rec2 = df_combined.merge(breed, on=["SE_Number"], how="left") - when cow database is used
col_keep = ["FarmName_Pseudo", "SE_Number", "BreedName", "LactationNumber", "CalvingDateDelPro", "upper_limit",
            "CullDecisionDate", "CullReason1", "CullReason2", "DryOffDate"]
for_my_rec2 = for_my_rec2[col_keep]
for_my_rec2.rename(columns={"BreedName": "BreedNameDelPro", "LactationNumber": "LactationNumberDelPro",
                            "BirthDate": "BirthDateDelPro",
                            "upper_limit": "UpperLimitDelPro", "CullDecisionDate": "CullDecisionDateDelPro",
                            "CullReason1": "CullReason1DelPro", "CullReason2": "CullReason2DelPro",
                            "DryOffDate": "DryOffDateDelPro"}, inplace=True)
for_my_rec2.to_csv("dfDelPro.csv", index=False)

In [ ]:
#
#
# ADD INSEMINATION DATA FROM DELPRO
# Load data, keep cowid, insdate, check for duplicates, sort
dfins = pd.read_csv("Del_Insemination240823.csv", delimiter=';', low_memory=False)
col_keep = ["FarmName_Pseudo", "SE_Number", "InseminationDate", "Breeder"]
dfins2 = dfins[col_keep]
print(dfins2.shape)  # 18,775 insemination events, 6col
dfins2 = dfins2.drop_duplicates(subset=["SE_Number", "InseminationDate"])
print(dfins2.shape)  # 18,689 unique insemination events, 6col
dfins2 = dfins2.sort_values(by=["SE_Number", "InseminationDate"])

dfins2.rename(columns={"InseminationDate": "InseminationDateDelPro", "Breeder": "BreederDelPro"}, inplace=True)

# Merge with dataframe
delpro = pd.read_csv("dfDelPro.csv", low_memory=False)
dfins4 = pd.merge(delpro, dfins2, on=["FarmName_Pseudo", "SE_Number"])
"""
# Subset chosen cow
SE_Number = ["SE-064c0cec-1189"]
dfins4 = dfins4[dfins4["SE_Number"].isin(SE_Number)]
"""
# Filter df for relevant insemination sorted to correct lactation
dfins5 = dfins4[(dfins4["InseminationDateDelPro"] >= dfins4["CalvingDateDelPro"]) & (dfins4["InseminationDateDelPro"] <= dfins4["UpperLimitDelPro"])]
dfins5.to_csv("dfDelPro.csv", index=False)

In [ ]:
#
#
# ADD PREGNANCY CHECKS FROM DELPRO
# Make next_ins to sort pregnancy checks
df20 = pd.read_csv("dfDelPro.csv", low_memory=False)
df20['next_ins'] = df20.groupby(['SE_Number', 'LactationNumberDelPro'])['InseminationDateDelPro'].shift(-1)

# only keep next_ins where falls within range
dfins7 = df20[(df20["next_ins"] >= df20["InseminationDateDelPro"]) & (df20["next_ins"] <= df20["UpperLimitDelPro"])]
col_keep = ["SE_Number", "LactationNumberDelPro", "InseminationDateDelPro", "next_ins", "UpperLimitDelPro"]
dfins7 = dfins7[col_keep]
dfins7.to_csv("dfDelPro2.csv", index=False)

"""
#Subset chosen cow
SE_Number = ["SE-064c0cec-1189"]
dfins8 = dfins8[dfins8["SE_Number"].isin(SE_Number)]
#dfins8.to_csv("dataframe3.csv", index=False)
"""

# Load pregnancy check data, check for duplicates, sort
preg = pd.read_csv("Del_PregnancyCheck240823.csv", delimiter=';', low_memory=False)

col_keep = ["FarmName_Pseudo", "SE_Number", "PregnancyCheckDate", "PregnancyCheckResult"]
preg = preg[col_keep]
preg.rename(columns={"PregnancyCheckDate": "PregnancyCheckDateDelPro",
                     "PregnancyCheckResult": "PregnancyCheckResultDelPro"}, inplace=True)

print(f"No. pregnancy checks in cow database: {preg.shape}")  # 14,169 events, 4col
preg = preg.drop_duplicates(subset=["SE_Number", "PregnancyCheckDateDelPro"])
print(f"No. unique pregnancy checks in cow database: {preg.shape}")    # 14,146 unique events, 4col
preg = preg.sort_values(by=["SE_Number", "PregnancyCheckDateDelPro"])

# Add to subset df
col_keep = ["SE_Number", "LactationNumberDelPro", "InseminationDateDelPro", "next_ins", "UpperLimitDelPro"]
df20 = df20[col_keep]
df21 = df20.join(preg.set_index(["SE_Number"]), on=["SE_Number"])
df21.to_csv("dfDelPro2.csv", index=False)

# Ensure datetime conversion
df21['InseminationDateDelPro'] = pd.to_datetime(df21['InseminationDateDelPro'])
df21['PregnancyCheckDateDelPro'] = pd.to_datetime(df21['PregnancyCheckDateDelPro'])
df21['next_ins'] = pd.to_datetime(df21['next_ins'])
df21['UpperLimitDelPro'] = pd.to_datetime(df21['UpperLimitDelPro'])

# Initialize 'C' column with NaN
df21['C'] = np.nan


# Define the filtering function
def filter_pregcheck(row):
    if pd.isna(row["next_ins"]):
        if (row["PregnancyCheckDateDelPro"] >= row["InseminationDateDelPro"]) and (row["PregnancyCheckDateDelPro"] <= row["UpperLimitDelPro"]):
            return "Yes"
        else:
            return "No"
    if pd.notna(row["next_ins"]):
        if (row["PregnancyCheckDateDelPro"] >= row["InseminationDateDelPro"]) and (row["PregnancyCheckDateDelPro"] <= row["next_ins"]):
            return "Yes"
        else:
            return "No"


# Apply the filter function to each row
df21['C'] = df21.apply(filter_pregcheck, axis=1)
dfins10 = df21[df21["C"] == "Yes"]
dfins10.to_csv("dfDelPro2.csv", index=False)

col_keep = ["SE_Number", "LactationNumberDelPro", "InseminationDateDelPro", "PregnancyCheckDateDelPro",
            "PregnancyCheckResultDelPro"]
dfins11 = dfins10[col_keep]

# Convert the 'InseminationDate' column from datetime64[ns] to object for merging
print(dfins11.dtypes)
dfins11 = dfins11.copy()
dfins11['InseminationDateDelPro'] = dfins11['InseminationDateDelPro'].astype(str)

# Add to master df
dfins12 = pd.read_csv("dfDelPro.csv")
print(dfins12.dtypes)
dfins13 = dfins12.merge(dfins11, on=["SE_Number", "LactationNumberDelPro", "InseminationDateDelPro"], how="left")
"""
#Subset chosen cow
SE_Number = ["SE-064c0cec-1189"]
dfins13 = dfins13[dfins13["SE_Number"].isin(SE_Number)]
"""
dfins13.to_csv("dfDelPro.csv", index=False)

# PAIRING COW DATABASE AND DELPRO DATA
- Use primarily cow database data, fill from DelPro where have missing data.

Creates the following datasets:
- calving.csv
- culling.csv
- dry_off.csv
- breed.csv
- insemination.csv
- pregnancy_checks.csv
- updateDF.csv

In [ ]:
#
#
# PAIRING COW DATABASE AND DELPRO DATA FOR CALVING
cKok = pd.read_csv("dfKok.csv", low_memory=False)
cKok["CalvingDate"] = cKok["CalvingDateKok"]
col_keep = ["FarmName_Pseudo", "SE_Number", "CalvingDate", "CalvingDateKok", "LactationNumberKok"]
cKok = cKok[col_keep]
cKok = cKok.drop_duplicates(subset=["SE_Number", "CalvingDateKok"])

cDel = pd.read_csv("dfDelPro.csv", low_memory=False)
cDel["CalvingDate"] = cDel["CalvingDateDelPro"]
col_keep = ["FarmName_Pseudo", "SE_Number", "CalvingDate", "CalvingDateDelPro", "LactationNumberDelPro"]
cDel = cDel[col_keep]
cDel = cDel.drop_duplicates(subset=["SE_Number", "CalvingDateDelPro"])

df_sum = pd.merge(cKok, cDel, on=["FarmName_Pseudo", "SE_Number", "CalvingDate"], how='outer')
df_sum = df_sum.sort_values(by=["FarmName_Pseudo", "SE_Number", "CalvingDate"])

"""
# Subset chosen cow - stämmer!
# SE-169e580a-3418 has 5 lactations, 4 in cow database (missing lact 3), last three in DelPro
SE_Number = ["SE-064c0cec-1189"]
df_sum = df_sum[df_sum["SE_Number"].isin(SE_Number)]
"""

df_sum['LactationNumber'] = df_sum['LactationNumberKok'].fillna(df_sum['LactationNumberDelPro'])
col_keep = ["FarmName_Pseudo", "SE_Number", "CalvingDate", "LactationNumber"]
df_sum = df_sum[col_keep]
df_sum.to_csv("calving.csv", index=False)

In [ ]:
#
#
# PAIRING COW DATABASE AND DELPRO FOR CULLING
cKok2 = pd.read_csv("dfKok.csv", low_memory=False)
cKok2 = cKok2.drop_duplicates(subset=["SE_Number", "ExitDateKok"])
cKok2["CullingDate"] = cKok2["ExitDateKok"]
col_keep = ["FarmName_Pseudo", "SE_Number", "CullingDate", "ExitDateKok", "ExitReason_PrimaryReasonKok",
            "ExitReason_SecondaryReason1Kok", "ExitReason_SecondaryReason2Kok"]
cKok2 = cKok2[col_keep]
cKok2 = cKok2.drop_duplicates(subset=["SE_Number", "CullingDate"])

cDel2 = pd.read_csv("for_my_rec2.csv", low_memory=False)
cDel2 = cDel2.drop_duplicates(subset=["SE_Number", "CullDecisionDateDelPro"])
cDel2["CullingDate"] = cDel2["CullDecisionDateDelPro"]
col_keep = ["FarmName_Pseudo", "SE_Number", "CullingDate", "CullDecisionDateDelPro", "CullReason1DelPro",
            "CullReason2DelPro"]
cDel2 = cDel2[col_keep]
cDel2 = cDel2.drop_duplicates(subset=["SE_Number", "CullingDate"])

cKok2['CullingDate'] = cKok2['ExitDateKok'].fillna(cDel2['CullDecisionDateDelPro'])
cKok2['CullingReason1'] = cKok2['ExitReason_PrimaryReasonKok'].fillna(cDel2['CullReason1DelPro'])
cKok2['CullingReason2'] = cKok2['ExitReason_SecondaryReason1Kok'].fillna(cDel2['CullReason2DelPro'])

"""
# Subset chosen cow
SE_Number = ["SE-064c0cec-1189"]
df_sum = df_sum[df_sum["SE_Number"].isin(SE_Number)]
"""

# filt = cKok[cKok["ExitDateKok"].isna()]
# filt = cKok[cKok["ExitReason_SecondaryReason2Kok"].notna()]

cKok2.drop(columns=["ExitDateKok"], inplace=True)
cKok2.to_csv("culling.csv", index=False)

In [ ]:
#
#
# PAIRING COW DATABASE AND DELPRO FOR DRY OFF
cKok3 = pd.read_csv("dfKok.csv", low_memory=False)
cKok3 = cKok3.drop_duplicates(subset=["SE_Number", "DryOffDateKok"])
col_keep = ["FarmName_Pseudo", "SE_Number", "LactationNumberKok", "DryOffDateKok"]
cKok3 = cKok3[col_keep]
cKok3.rename(columns={"LactationNumberKok": "LactationNumber"}, inplace=True)
cKok3 = cKok3.drop_duplicates(subset=["SE_Number", "DryOffDateKok"])

cDel3 = pd.read_csv("for_my_rec2.csv", low_memory=False)
cDel3 = cDel3.drop_duplicates(subset=["SE_Number", "DryOffDateDelPro"])
col_keep = ["FarmName_Pseudo", "SE_Number", "LactationNumberDelPro", "DryOffDateDelPro"]
cDel3 = cDel3[col_keep]
cDel3.rename(columns={"LactationNumberDelPro": "LactationNumber"}, inplace=True)
cDel3 = cDel3.drop_duplicates(subset=["SE_Number", "DryOffDateDelPro"])

merged_df = pd.merge(cKok3, cDel3, on=["FarmName_Pseudo", "SE_Number", "LactationNumber"], how='outer')
merged_df['DryOffDate'] = merged_df['DryOffDateKok'].combine_first(merged_df['DryOffDateDelPro'])
merged_df = merged_df.sort_values(by=["FarmName_Pseudo", "SE_Number", "LactationNumber"])

"""
# Subset chosen cow
# obs 867, SE-169e580a-2843, good example cow
SE_Number = ["SE-064c0cec-1189"]
merged_df = merged_df[merged_df["SE_Number"].isin(SE_Number)]
"""

merged_df.to_csv("dry_off.csv", index=False)

In [ ]:
#
#
# PAIRING COW DATABASE AND DELPRO FOR BREED
cKok4 = pd.read_csv("dfKok.csv", low_memory=False)
cKok4 = cKok4.drop_duplicates(subset=["SE_Number"])
col_keep = ["FarmName_Pseudo", "SE_Number", "SireBreedKok", "DamBreedKok", "MGSBreedKok"]
cKok4 = cKok4[col_keep]
cKok4 = cKok4.drop_duplicates(subset=["SE_Number"])

cDel4 = pd.read_csv("for_my_rec2.csv", low_memory=False)
cDel4 = cDel4.drop_duplicates(subset=["SE_Number"])
col_keep = ["FarmName_Pseudo", "SE_Number", "BreedNameDelPro"]
cDel4 = cDel4[col_keep]
cDel4 = cDel4.drop_duplicates(subset=["SE_Number"])

merged_df = pd.merge(cKok4, cDel4, on=["FarmName_Pseudo", "SE_Number"], how='outer')

# Setting options to display more rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

frequency_table = pd.crosstab(merged_df['SireBreedKok'], merged_df['DamBreedKok'])
print(frequency_table)


# Define breeds in cow database
def categorize1(value):
    if (value["SireBreedKok"] == "SRB") and (value["DamBreedKok"] == "SRB"):
        return "NRDC"
    if (value["SireBreedKok"] == "RB") and (value["DamBreedKok"] == "RB"):
        return "NRDC"
    if (value["SireBreedKok"] == "RB-SRB") and (value["DamBreedKok"] == "RB-SRB"):
        return "NRDC"
    if (value["SireBreedKok"] == "RB") and (value["DamBreedKok"] == "RB-SRB"):
        return "NRDC"
    if (value["SireBreedKok"] == "SRB") and (value["DamBreedKok"] == "RB"):
        return "NRDC"
    if (value["SireBreedKok"] == "RB-SRB") and (value["DamBreedKok"] == "RB-SRB"):
        return "NRDC"
    if (value["SireBreedKok"] == "SRB") and (value["DamBreedKok"] == "RB-SRB"):
        return "NRDC"
    if (value["SireBreedKok"] == "RB") and (value["DamBreedKok"] == "SRB"):
        return "NRDC"
    if (value["SireBreedKok"] == "RB-SRB") and (value["DamBreedKok"] == "SRB"):
        return "NRDC"
    if (value["SireBreedKok"] == "SRB-RB") and (value["DamBreedKok"] == "SRB"):
        return "NRDC"
    if (value["SireBreedKok"] == "RB") and (value["DamBreedKok"] == "SRB-RB"):
        return "NRDC"
    if (value["SireBreedKok"] == "SAB") and (value["DamBreedKok"] == "SAB"):
        return "NRDC"

    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SLB"):
        return "SLB"
    if (value["SireBreedKok"] == "SJB") and (value["DamBreedKok"] == "SJB") and (value["MGSBreedKok"] == "SJB"):
        return "SJB"

    if (value["SireBreedKok"] == "FJÄ") and (value["DamBreedKok"] == "FJÄ"):
        return "Other"
    if (value["SireBreedKok"] == "SIM") and (value["DamBreedKok"] == "SIM"):
        return "Other"

    if (value["SireBreedKok"] == "SRB") and (value["DamBreedKok"] == "SLB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SRB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "RB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "RB-SLB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "RB-SRB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SJB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SJB-SLB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SJB-SRB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SKB-SJB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SKB-SLB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SLB-RB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SLB-SJB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SLB-SKB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SLB-SRB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SRB-RB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SRB-SJB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SRB-SLB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SRB") and (value["DamBreedKok"] == "RB-SJB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SRB") and (value["DamBreedKok"] == "SJB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SRB") and (value["DamBreedKok"] == "SJB-SLB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SRB") and (value["DamBreedKok"] == "SJB-SRB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SRB") and (value["DamBreedKok"] == "SLB-SRB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SRB") and (value["DamBreedKok"] == "SRB-SKB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SRB") and (value["DamBreedKok"] == "SRB-SLB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "RB") and (value["DamBreedKok"] == "SJB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "RB") and (value["DamBreedKok"] == "SJB-SRB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "RB") and (value["DamBreedKok"] == "SLB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "RB") and (value["DamBreedKok"] == "SRB-SJB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "RB") and (value["DamBreedKok"] == "SRB-SLB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SKB") and (value["DamBreedKok"] == "FJÄ"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SRB") and (value["DamBreedKok"] == "SAB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SAB") and (value["DamBreedKok"] == "SAB-SRB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SAB-SRB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SRB") and (value["DamBreedKok"] == "SAB-SRB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SAB-SRB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SLB-BSW"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SLB-SAB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "RB") and (value["DamBreedKok"] == "SRB-SAB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SAB") and (value["DamBreedKok"] == "SRB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SRB-SAB"):
        return "DairyCross"
    if (value["SireBreedKok"] == "SRB") and (value["DamBreedKok"] == "SRB-SAB"):
        return "DairyCross"

    if (value["SireBreedKok"] == "RB") and (value["DamBreedKok"] == "MON-SLB"):
        return "DairyBeefCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "CHA-RB"):
        return "DairyBeefCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "HER-SLB"):
        return "DairyBeefCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "MON-RB"):
        return "DairyBeefCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "MON-SLB"):
        return "DairyBeefCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "MON-SRB"):
        return "DairyBeefCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SLB-HER"):
        return "DairyBeefCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SRB-MON"):
        return "DairyBeefCross"
    if (value["SireBreedKok"] == "SLB") and (value["DamBreedKok"] == "SLB-MON"):
        return "DairyBeefCross"
    if (value["SireBreedKok"] == "SRB") and (value["DamBreedKok"] == "MON-SLB"):
        return "DairyBeefCross"
    if (value["SireBreedKok"] == "SRB") and (value["DamBreedKok"] == "MON-SRB"):
        return "DairyBeefCross"
    if (value["SireBreedKok"] == "SJB") and (value["DamBreedKok"] == "SIM-SJB"):
        return "DairyBeefCross"
    if (value["SireBreedKok"] == "SJB") and (value["DamBreedKok"] == "SJB-SIM"):
        return "DairyBeefCross"

    if (value["SireBreedKok"] == "CHA") and (value["DamBreedKok"] == "RB-SRB"):
        return "BeefDairyCross"
    if (value["SireBreedKok"] == "CHA") and (value["DamBreedKok"] == "SLB-SRB"):
        return "BeefDairyCross"
    if (value["SireBreedKok"] == "MON") and (value["DamBreedKok"] == "SLB"):
        return "BeefDairyCross"
    if (value["SireBreedKok"] == "MON") and (value["DamBreedKok"] == "SRB"):
        return "BeefDairyCross"
    if (value["SireBreedKok"] == "MON") and (value["DamBreedKok"] == "RB-SLB"):
        return "BeefDairyCross"
    if (value["SireBreedKok"] == "MON") and (value["DamBreedKok"] == "RB-SRB"):
        return "BeefDairyCross"
    if (value["SireBreedKok"] == "MON") and (value["DamBreedKok"] == "SRB-SLB"):
        return "BeefDairyCross"
    if (value["SireBreedKok"] == "MON") and (value["DamBreedKok"] == "SLB-SRB"):
        return "BeefDairyCross"
    if (value["SireBreedKok"] == "SIM") and (value["DamBreedKok"] == "SJB"):
        return "BeefDairyCross"
    if (value["SireBreedKok"] == "SIM") and (value["DamBreedKok"] == "SRB"):
        return "BeefDairyCross"
    if (value["SireBreedKok"] == "SIM") and (value["DamBreedKok"] == "SRB-SJB"):
        return "BeefDairyCross"
    if (value["SireBreedKok"] == "SIM") and (value["DamBreedKok"] == "SRB-SAB"):
        return "BeefDairyCross"
    if (value["SireBreedKok"] == "HER") and (value["DamBreedKok"] == "SLB-SRB"):
        return "BeefDairyCross"

    if pd.isna(value["SireBreedKok"]) and pd.isna(value["DamBreedKok"]) and pd.isna(value["MGSBreedKok"]):
        return np.nan
    else:
        return 'Other'


merged_df['BreedKok'] = merged_df.apply(categorize1, axis=1)
merged_df.to_csv("breed.csv", index=False)

# Count the occurrences of each unique value in the breed column
value_counts = merged_df['BreedKok'].value_counts()
plt.figure(figsize=(10, 10))  # Optional: Set the figure size
plt.pie(value_counts, labels=value_counts.index, autopct='%1.1f%%', startangle=140, pctdistance=0.85)
plt.axis('equal')
plt.title('Distribution of Breeds of Individuals According to Cow Database')
plt.show()

# Fix missing breed info from DelPro
frequency_table = merged_df['BreedNameDelPro'].value_counts()
print(frequency_table)


# Define breeds in delpro
def categorize2(value):
    if value["BreedNameDelPro"] == "01 SRB":
        return "NRDC"
    if value["BreedNameDelPro"] == "06 RB":
        return "NRDC"
    if value["BreedNameDelPro"] == "02 SLB":
        return "SLB"
    if value["BreedNameDelPro"] == "04 SJB":
        return "SJB"
    if value["BreedNameDelPro"] == "03 SKB":
        return "Other"
    if value["BreedNameDelPro"] == "Unknown Breed":
        return "Unknown"
    if value["BreedNameDelPro"] == "186":
        return "Unknown"
    if value["BreedNameDelPro"] == "187":
        return "Unknown"
    if value["BreedNameDelPro"] == "99 Korsning/Obest Ras":
        return "Unknown"
    if value["BreedNameDelPro"] == "99 Korsning/övriga raser":
        return "Unknown"
    if value["BreedNameDelPro"] == "41 Fjällko":
        return "Other"
    if value["BreedNameDelPro"] == "08 Hereford":
        return "Other"
    if value["BreedNameDelPro"] == "11 Aberdeen Angus":
        return "Other"
    if value["BreedNameDelPro"] == "28 Fleckvieh":
        return "Other"
    if value["BreedNameDelPro"] == "27 Montbéliard":
        return "Other"
    if (value["BreedKok"] == "Missing") & pd.isna(value["BreedNameDelPro"]):
        return np.nan
    if pd.isna(value["BreedNameDelPro"]):
        return np.nan
    else:
        return "Other"


merged_df['BreedDelPro'] = merged_df.apply(categorize2, axis=1)

merged_df['Breed'] = merged_df['BreedKok'].fillna(merged_df['BreedDelPro'])
merged_df.to_csv("breed.csv", index=False)

# Count the occurrences of each unique value in the 'Fruit' column
value_counts = merged_df['Breed'].value_counts()
plt.figure(figsize=(10, 10))
plt.pie(value_counts, labels=value_counts.index, autopct='%1.1f%%', startangle=140, pctdistance=0.85)
plt.axis('equal')
plt.title('Distribution of Breeds in Full Material')
plt.show()

# Fix missing breed info from DelPro
frequency_table = merged_df['Breed'].value_counts()
print(frequency_table)

In [ ]:
#
#
# PAIRING COW DATABASE AND DELPRO FOR INSEMINATION DATA
cKok4 = pd.read_csv("dfKok.csv", low_memory=False)
cKok4 = cKok4.drop_duplicates(subset=["SE_Number", "InseminationDateKok"])
col_keep = ["FarmName_Pseudo", "SE_Number", "LactationNumberKok", "InseminationDateKok"]
cKok4 = cKok4[col_keep]
cKok4.rename(columns={"LactationNumberKok": "LactationNumber"}, inplace=True)
cKok4 = cKok4.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDateKok"])

cDel4 = pd.read_csv("dfDelPro.csv", low_memory=False)
cDel4 = cDel4.drop_duplicates(subset=["SE_Number", "InseminationDateDelPro"])
col_keep = ["FarmName_Pseudo", "SE_Number", "LactationNumberDelPro", "InseminationDateDelPro"]
cDel4 = cDel4[col_keep]
cDel4.rename(columns={"LactationNumberDelPro": "LactationNumber"}, inplace=True)
cDel4 = cDel4.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDateDelPro"])

merged_df = pd.merge(cKok4, cDel4, on=["FarmName_Pseudo", "SE_Number", "LactationNumber"], how='outer')
merged_df['InseminationDate'] = merged_df['InseminationDateKok'].fillna(merged_df['InseminationDateDelPro'])
merged_df = merged_df.drop_duplicates(subset=["SE_Number", "InseminationDate"])

"""
# Subset chosen cow
# obs 867, SE-169e580a-2843, good example cow
SE_Number = ["SE-064c0cec-1189"]
merged_df = merged_df[merged_df["SE_Number"].isin(SE_Number)]
"""

merged_df = merged_df.sort_values(by=["FarmName_Pseudo", "SE_Number", "LactationNumber", "InseminationDate"])
merged_df.to_csv("insemination.csv", index=False)

In [ ]:
#
#
# PAIRING PREGNANCY CHECK FROM COW DATABASE AND DELPRO
cKok5 = pd.read_csv("dfKok.csv", low_memory=False)
cKok5 = cKok5.drop_duplicates(subset=["SE_Number", "PregnancyCheckDateKok"])
col_keep = ["FarmName_Pseudo", "SE_Number", "LactationNumberKok", "PregnancyCheckDateKok", "PregnancyStatusKok"]
cKok5 = cKok5[col_keep]
cKok5.rename(columns={"LactationNumberKok": "LactationNumber"}, inplace=True)
cKok5 = cKok5.drop_duplicates(subset=["SE_Number", "LactationNumber", "PregnancyCheckDateKok"])

cDel5 = pd.read_csv("dfDelPro.csv", low_memory=False)
cDel5 = cDel5.drop_duplicates(subset=["SE_Number", "InseminationDateDelPro"])
col_keep = ["FarmName_Pseudo", "SE_Number", "LactationNumberDelPro", "PregnancyCheckDateDelPro",
            "PregnancyCheckResultDelPro"]
cDel5 = cDel5[col_keep]
cDel5.rename(columns={"LactationNumberDelPro": "LactationNumber"}, inplace=True)
cDel5 = cDel5.drop_duplicates(subset=["SE_Number", "LactationNumber", "PregnancyCheckDateDelPro"])

merged_df = pd.merge(cKok5, cDel5, on=["FarmName_Pseudo", "SE_Number", "LactationNumber"], how='outer')
merged_df['PregnancyCheckDate'] = merged_df['PregnancyCheckDateKok'].fillna(merged_df['PregnancyCheckDateDelPro'])
merged_df['PregnancyStatus'] = merged_df['PregnancyStatusKok'].fillna(merged_df['PregnancyCheckResultDelPro'])
merged_df = merged_df.drop_duplicates(subset=["SE_Number", "PregnancyCheckDate"])

"""
# Subset chosen cow
# obs 867, SE-169e580a-2843, good example cow
SE_Number = ["SE-064c0cec-1189"]
merged_df = merged_df[merged_df["SE_Number"].isin(SE_Number)]
"""

merged_df.to_csv("pregnancy_checks.csv", index=False)

In [ ]:
#
#
# BUILD MASTER DATAFRAME
test2a = pd.read_csv("calving.csv", low_memory=False)
test2b = pd.read_csv("breed.csv", low_memory=False)
col_keep = ["FarmName_Pseudo", "SE_Number", "Breed"]
test2b = test2b[col_keep]
test2c = pd.merge(test2a, test2b, on=["FarmName_Pseudo", "SE_Number"], how='outer')
test2c.to_csv("updateDF.csv", index=False)

test2d = pd.read_csv("insemination.csv", low_memory=False)
col_keep = ["FarmName_Pseudo", "SE_Number", "LactationNumber", "InseminationDate"]
test2d = test2d[col_keep]
test2d = test2d.drop_duplicates(subset=["FarmName_Pseudo", "SE_Number", "LactationNumber", "InseminationDate"])
test2e = pd.merge(test2c, test2d, on=["FarmName_Pseudo", "SE_Number", "LactationNumber"], how='outer')
test2e.to_csv("updateDF.csv", index=False)

test2f = pd.read_csv("pregnancy_checks.csv", low_memory=False)
col_keep = ["FarmName_Pseudo", "SE_Number", "LactationNumber", "PregnancyCheckDate", "PregnancyStatus"]
test2f = test2f[col_keep]
test2f = test2f.drop_duplicates(subset=["FarmName_Pseudo", "SE_Number", "LactationNumber", "PregnancyCheckDate"])
test2g = pd.merge(test2e, test2f, on=["FarmName_Pseudo", "SE_Number", "LactationNumber"], how='outer')
# test2g = test2g.drop_duplicates(subset=["FarmName_Pseudo", "SE_Number", "LactationNumber", "PregnancyCheckDate"])
test2g = test2g.sort_values(by=["FarmName_Pseudo", "SE_Number", "LactationNumber", "InseminationDate", "PregnancyCheckDate"])
test2g.to_csv("updateDF.csv", index=False)

test2h = pd.read_csv("dry_off.csv", low_memory=False)
col_keep = ["FarmName_Pseudo", "SE_Number", "LactationNumber", "DryOffDate"]
test2h = test2h[col_keep]
test2h = test2h.drop_duplicates(subset=["FarmName_Pseudo", "SE_Number", "LactationNumber", "DryOffDate"])
test2i = pd.merge(test2g, test2h, on=["FarmName_Pseudo", "SE_Number", "LactationNumber"], how='outer')

"""
# Subset chosen cow
# obs 867, SE-169e580a-2843, good example cow
SE_Number = ["SE-064c0cec-1189"]
test2i = test2i[test2i["SE_Number"].isin(SE_Number)]
"""
test2i.to_csv("updateDF.csv", index=False)

test2j = pd.read_csv("culling.csv", low_memory=False)
test2j = test2j.drop_duplicates(subset=["FarmName_Pseudo", "SE_Number", "CullingDate"])
test2k = pd.merge(test2i, test2j, on=["FarmName_Pseudo", "SE_Number"], how='outer')
test2k.to_csv("updateDF.csv", index=False)

"""
# Subset chosen cow
# obs 867, SE-169e580a-2843, good example cow
SE_Number = ["SE-064c0cec-1189"]
test2k = test2k[test2k["SE_Number"].isin(SE_Number)]
"""

col_keep = ["SE_Number", "LactationNumber", "CalvingDate", "InseminationDate", "PregnancyCheckDate", "PregnancyStatus"]
test2k2 = test2k[col_keep]

# Sort pregnancy checks and keep only for relevant insemination
test2l = test2k2.drop_duplicates(subset=["SE_Number", "CalvingDate", "InseminationDate"])
test2l = test2l.copy()
test2l["next_ins"] = test2l.groupby(["SE_Number", "LactationNumber"])["InseminationDate"].shift(-1)
col_keep = ["SE_Number", "LactationNumber", "InseminationDate", "next_ins"]
test2l = test2l[col_keep]
test2m = pd.merge(test2k2, test2l, on=["SE_Number", "LactationNumber", "InseminationDate"], how='outer')
# test2m.to_csv("updateDF2.csv", index=False)

test2m.loc[test2m['PregnancyCheckDate'] > test2m['next_ins'], 'PregnancyCheckDate'] = np.nan
test2m.loc[test2m['PregnancyCheckDate'] < test2m['InseminationDate'], 'PregnancyCheckDate'] = np.nan
# test2m.to_csv("updateDF3.csv", index=False)

test2n = test2m.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate", "PregnancyCheckDate"])
test2n = test2n.sort_values(by=["SE_Number", "LactationNumber", "InseminationDate", "PregnancyCheckDate"])
# test2n.to_csv("updateDF.csv", index=False)

df_not_missing = test2n[test2n['PregnancyCheckDate'].notna()]
# df_not_missing.to_csv("updateDF4.csv", index=False)

test2o = test2n[test2n['PregnancyCheckDate'].isnull()]
col_keep = ["SE_Number", "LactationNumber", "CalvingDate", "InseminationDate"]
test2o = test2o[col_keep]
# test2o.to_csv("updateDF5.csv", index=False)

test2p = pd.merge(df_not_missing, test2o, on=["SE_Number", "LactationNumber", "CalvingDate", "InseminationDate"], how='outer')
# test2p.to_csv("updateDF2.csv", index=False)

test2q = pd.read_csv("updateDF.csv", low_memory=False)
col_keep = ["FarmName_Pseudo", "SE_Number", "LactationNumber", "CalvingDate", "Breed", "DryOffDate", "CullingDate",
            "ExitReason_PrimaryReasonKok", "ExitReason_SecondaryReason1Kok", "ExitReason_SecondaryReason2Kok",
            "CullingReason1", "CullingReason2"]
test2q = test2q[col_keep]
test2r = pd.merge(test2q, test2p, on=["SE_Number", "LactationNumber", "CalvingDate"], how='left')
test2r.to_csv("updateDF3.csv", index=False)

df = pd.read_csv("updateDF3.csv", low_memory=False)
print(f"No. of pregnancy checks in database: {df.shape}")  # 327,839 pregnancy checks
df2 = df.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate"])
print(f"No. of inseminations in dataset: {df2.shape}")  # 36,506 inseminations
df2 = df.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. of lactations in dataset: {df2.shape}")  # 20,683 lactations
df2 = df.drop_duplicates(subset=["SE_Number"])
print(f"No. of cows in dataset: {df2.shape}")  # 9,535 cows
df2 = df.drop_duplicates(subset=["FarmName_Pseudo"])
print(f"No. of herds in dataset: {df2.shape}")  # 57 herds

#
#
# ADD LINEAGE INFORMATION
lin = pd.read_csv("Kok_Lineage240821.csv", delimiter=';', low_memory=False)
col_keep = ["BirthID", "BirthDate", "Father_SE_Number", "Mother_SE_Number"]
lin = lin[col_keep]
lin.rename(columns={'BirthID': 'SE_Number'}, inplace=True)

print(f"No. of unique cows in database: {lin.shape}")  # 24,067
lin = lin.drop_duplicates(subset=["SE_Number", "BirthDate", "Father_SE_Number", "Mother_SE_Number"])
print(f"No. of unique cows in database: {lin.shape}")  # 22,666

df = pd.read_csv("updateDF3.csv", low_memory=False)
lin2 = df.merge(lin, on=["SE_Number"], how="left")

# Change order of columns
new_column_order = ["FarmName_Pseudo", "SE_Number", "Breed", "BirthDate", "Father_SE_Number", "Mother_SE_Number",
                    "CalvingDate", "LactationNumber", "InseminationDate",
                    "PregnancyCheckDate", "PregnancyStatus", "DryOffDate", "CullingDate", "ExitReason_PrimaryReasonKok",
                    "ExitReason_SecondaryReason1Kok", "ExitReason_SecondaryReason2Kok", "CullingReason1",
                    "CullingReason2"]
lin2 = lin2[new_column_order]
lin2 = lin2.drop_duplicates(subset=["SE_Number", "CalvingDate", "LactationNumber", "InseminationDate",
                                    "PregnancyCheckDate"])
"""
SE_Number = ["SE-064c0cec-1189"]
lin2 = lin2[lin2["SE_Number"].isin(SE_Number)]
"""

# REORDER PregnancyStatus
unique_values = lin2['PregnancyStatus'].unique()
print(unique_values)

lin2["PregnancyStatus"] = lin2["PregnancyStatus"].replace(
    {"Dräktig (undersökt) Dr": 2,
     "Dräktig Analys Dr A": 52,
     "Dräktig (ej undersökt) Dr": 22,
     "Dräktig (tjurbetäckt) Dr": 32,
     "Ej dräktig (tjurbetäckt) eDr": 31,
     "Dräktig ? Analys Dr? A": 53,
     "Ej Dräktig (ej undersökt) eDr": 21,
     "Negative": 51,
     "Positive": 52,
     "Uncertain": 53,
     "Dräktig ? (undersökt) Dr?": 3,
     "Dräktig, (sem/bet annan bes) Dr": 42,
     "Dräktig ? (tjurbetäckt) Dr?": 33,
     "Ej dräktig Analys eDr A": 51,
     "Ej Dräktig (undersökt) eDr": 1})

lin2.to_csv("updateDF.csv", index=False)

# MILKING DATA FROM DELPRO
- using Del_CowMilkYield_Common
- creates MY.csv

In [ ]:
#
#
# MILKING DATA
# Del_CowMilkYield_Common240823
dfmy = pd.read_csv("Del_CowMilkYield_Common240823.csv", delimiter=';', low_memory=False)
print(f"No. of milking events in raw data: {dfmy.shape}")  # 5,628,715 x 15
print(dfmy.dtypes)

# No. lactation and cows in raw data
test = dfmy.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. of lactation in raw milking file: {test.shape}")  # 8,352
test = dfmy.drop_duplicates(subset=["SE_Number"])
print(f"No. of cows in raw milking file: {test.shape}")  # 3,916

# Remove pure duplicates
dfmy = dfmy.drop_duplicates(subset=["SE_Number", "LactationNumber", "StartDate", "StartTime", "SessionNumber", "TotalYield"])
print(f"No. of milking events in raw data: {dfmy.shape}")  # 5,624,788

# No. lactation and cows in data
test = dfmy.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. of lactation after removed pure duplicates: {test.shape}")  # 8,352
test = test.drop_duplicates(subset=["SE_Number"])
print(f"No. of cows after removed pure duplicates {test.shape}")  # 3,916

# Keep relevant col
# LactationNumber removed because of missing values, e.g. SE-064c0cec-1188
# DaysInMilk removed due to counting day 1 as day 0, reinitialize later
col_keep = ["FarmName_Pseudo", "SE_Number", "LactationNumber", "DaysInMilk", "StartDate", "StartTime", "SessionNumber", "TotalYield"]
dfmy = dfmy[col_keep]

# Which herds are in this file?
dfmy_unique2 = dfmy.drop_duplicates(subset=["FarmName_Pseudo"])
dfmy_unique2 = dfmy_unique2.drop(columns=["SE_Number", "StartDate", "StartTime", "LactationNumber", "DaysInMilk", "SessionNumber", "TotalYield"])
print(dfmy_unique2.to_string(index=False))

# Change TotalYield comma to dot
dfmy["TotalYield"] = dfmy["TotalYield"].str.replace(',', '.')
# Change to datetime
dfmy["StartDate"] = pd.to_datetime(dfmy["StartDate"])

dfmy = dfmy.sort_values(by=["SE_Number", "LactationNumber", "StartDate", "StartTime", "SessionNumber", "TotalYield"])

# Remove missing yield
dfmy2 = dfmy[pd.notna(dfmy["TotalYield"])]
print(f"No. of milking records with recorded yield in milking file: {dfmy2.shape}")  # 4,948,464

# No. lactation and cows in data
test = dfmy2.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. of lactations after removed missing yield: {test.shape}")  # 7920
test = dfmy2.drop_duplicates(subset=["SE_Number"])
print(f"No. of cows after removed missing yield: {test.shape}")  # 3,657

# Remove duplicated sessions
dfmy2 = dfmy2.sort_values(by=['SE_Number', 'StartDate', 'SessionNumber'])
dfmy2 = dfmy2.drop_duplicates(subset=["SE_Number", "LactationNumber", "StartDate", "SessionNumber", "TotalYield"])
print(f"No. of unique milking records in milking file: {dfmy2.shape}")  # 4,947,223
dfmy2.to_csv("MY.csv", index=False)

# No. lactation and cows in data
test = dfmy2.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. of lactations after removed duplicated sessions: {test.shape}")  # 7,920
test = dfmy2.drop_duplicates(subset=["SE_Number"])
print(f"No. of cows after removed duplicated sessions: {test.shape}")  # 3,657

# Check raw data for each herd
FarmName_Pseudo = ["a624fb9a"]
lactation_yield = dfmy2[dfmy2["FarmName_Pseudo"].isin(FarmName_Pseudo)]
lactation_yield = lactation_yield.sort_values(by=['SE_Number', 'StartDate', 'SessionNumber'])
lactation_yield.to_csv("lact3a.csv", index=False)
print(f"No. of unique milking records in a624fb9a milking file: {lactation_yield.shape}")  # 441,149

FarmName_Pseudo = ["5f7f33d6"]
lactation_yield = dfmy2[dfmy2["FarmName_Pseudo"].isin(FarmName_Pseudo)]
lactation_yield = lactation_yield.sort_values(by=['SE_Number', 'StartDate', 'SessionNumber'])
lactation_yield.to_csv("lact3b.csv", index=False)
print(f"No. of unique milking records in 5f7f33d6 milking file: {lactation_yield.shape}")  # 485,361

FarmName_Pseudo = ["ab18b151"]
lactation_yield = dfmy2[dfmy2["FarmName_Pseudo"].isin(FarmName_Pseudo)]
lactation_yield = lactation_yield.sort_values(by=['SE_Number', 'StartDate', 'SessionNumber'])
lactation_yield.to_csv("lact3c.csv", index=False)
print(f"No. of unique milking records in ab18b151 milking file: {lactation_yield.shape}")  # 231,172

FarmName_Pseudo = ["f454e660"]
lactation_yield = dfmy2[dfmy2["FarmName_Pseudo"].isin(FarmName_Pseudo)]
lactation_yield = lactation_yield.sort_values(by=['SE_Number', 'StartDate', 'SessionNumber'])
lactation_yield.to_csv("lact3d.csv", index=False)
print(f"No. of unique milking records in f454e660 milking file: {lactation_yield.shape}")  # 449,080

FarmName_Pseudo = ["540275a1"]
lactation_yield = dfmy2[dfmy2["FarmName_Pseudo"].isin(FarmName_Pseudo)]
lactation_yield = lactation_yield.sort_values(by=['SE_Number', 'StartDate', 'SessionNumber'])
lactation_yield.to_csv("lact3e.csv", index=False)
print(f"No. of unique milking records in 540275a1 milking file: {lactation_yield.shape}")  # 312,961

FarmName_Pseudo = ["afdd9a78"]
lactation_yield = dfmy2[dfmy2["FarmName_Pseudo"].isin(FarmName_Pseudo)]
lactation_yield = lactation_yield.sort_values(by=['SE_Number', 'StartDate', 'SessionNumber'])
lactation_yield.to_csv("lact3f.csv", index=False)
print(f"No. of unique milking records in afdd9a78 milking file: {lactation_yield.shape}")  # 45,618

FarmName_Pseudo = ["5b581702"]
lactation_yield = dfmy2[dfmy2["FarmName_Pseudo"].isin(FarmName_Pseudo)]
lactation_yield = lactation_yield.sort_values(by=['SE_Number', 'StartDate', 'SessionNumber'])
lactation_yield.to_csv("lact3g.csv", index=False)
print(f"No. of unique milking records in 5b581702 milking file: {lactation_yield.shape}")  # 111,634

FarmName_Pseudo = ["5c06d92d"]
lactation_yield = dfmy2[dfmy2["FarmName_Pseudo"].isin(FarmName_Pseudo)]
lactation_yield = lactation_yield.sort_values(by=['SE_Number', 'StartDate', 'SessionNumber'])
lactation_yield.to_csv("lact3h.csv", index=False)
print(f"No. of unique milking records in 5c06d92d milking file: {lactation_yield.shape}")  # 971,965

FarmName_Pseudo = ["752efd72"]
lactation_yield = dfmy2[dfmy2["FarmName_Pseudo"].isin(FarmName_Pseudo)]
lactation_yield = lactation_yield.sort_values(by=['SE_Number', 'StartDate', 'SessionNumber'])
lactation_yield.to_csv("lact3i.csv", index=False)
print(f"No. of unique milking records in 752efd72 milking file: {lactation_yield.shape}")  # 863,077

# Have 300,000 records but almost all missing MY!
FarmName_Pseudo = ["a756bc39"]
lactation_yield = dfmy[dfmy["FarmName_Pseudo"].isin(FarmName_Pseudo)]
lactation_yield = lactation_yield.sort_values(by=['SE_Number', 'StartDate', 'SessionNumber'])
lactation_yield.to_csv("lact3j.csv", index=False)
print(f"No. of unique milking records in a756bc39 milking file: {lactation_yield.shape}")  # 314,841! MY miss

FarmName_Pseudo = ["ad0a39f5"]
lactation_yield = dfmy2[dfmy2["FarmName_Pseudo"].isin(FarmName_Pseudo)]
lactation_yield = lactation_yield.sort_values(by=['SE_Number', 'StartDate', 'SessionNumber'])
lactation_yield.to_csv("lact3k.csv", index=False)
print(f"No. of unique milking records in ad0a39f5 milking file: {lactation_yield.shape}")  # 517,188

# Have 138,922 sessions recorded, but almost all are missing yield!
FarmName_Pseudo = ["6d38bc90"]
lactation_yield = dfmy[dfmy["FarmName_Pseudo"].isin(FarmName_Pseudo)]
lactation_yield = lactation_yield.sort_values(by=['SE_Number', 'StartDate', 'SessionNumber'])
lactation_yield.to_csv("lact3l.csv", index=False)
print(f"No. of unique milking records in 6d38bc90 milking file: {lactation_yield.shape}")  # 153,211 MY miss!

# Find start of recording within respective herd
first_observations = dfmy2.groupby("FarmName_Pseudo")["StartDate"].first().reset_index()
print(f"Start of recording in Gigacow in different herds: \n", first_observations.to_string(index=False))

# No. MY records per herd
count_my_rec = dfmy2.groupby(["FarmName_Pseudo"])["StartDate"].count().reset_index()
print(f"No. milking events per herd: \n", count_my_rec.to_string(index=False))

# Add lactation number and upper_limit to MY file to sort MY data to correct lactation
# Because LactationNumber is not always used in the MY file
test2l = pd.read_csv("updateDF.csv", low_memory=False)
print(f"Calving, insemination, pregnancy checks, culling and dry off data with milking data: {test2l.shape}")
# 22,972,867

test2m = pd.read_csv("dfKok.csv", low_memory=False)
test2m = test2m.drop_duplicates(subset=["SE_Number", "LactationNumberKok", "upper_limit"])
test2m.rename(columns={"LactationNumberKok": "LactationNumber"}, inplace=True)
col_keep = ["SE_Number", "LactationNumber", "upper_limit"]
test2m = test2m[col_keep]

print(test2l.columns)
print(test2m.columns)

test2n = pd.merge(test2l, test2m, on=["SE_Number", "LactationNumber"], how='left')
col_keep = ["FarmName_Pseudo", "SE_Number", "CalvingDate", "LactationNumber", "upper_limit"]
test2n = test2n[col_keep]
test2n = test2n.drop_duplicates(subset=["SE_Number", "LactationNumber"])

"""
# Subset chosen cow
SE_Number = ["SE-064c0cec-1189"]
test2n = test2n[test2n["SE_Number"].isin(SE_Number)]
test2n.to_csv("MY2.csv", index=False)
"""

# Load MY data
dfmy3 = pd.read_csv("MY.csv", low_memory=False)
col_keep = ["FarmName_Pseudo", "SE_Number", "StartDate", "StartTime", "SessionNumber", "TotalYield"]
dfmy3 = dfmy3[col_keep]

"""
# Subset chosen cow
SE_Number = ["SE-064c0cec-1189"]
dfmy3 = dfmy3[dfmy3["SE_Number"].isin(SE_Number)]
"""

# Merge
dfmy3 = test2n.merge(dfmy3, on=["FarmName_Pseudo", "SE_Number"], how="left")
dfmy3 = dfmy3.sort_values(by=["SE_Number", "LactationNumber", "StartDate", "StartTime"])

# only keep MY data where falls within lactation, ie between CalvingDate and upper_limit
# where upper_limit is either next lactation, slaughter date or today's date for open records (ie ongoing lactation)
dfmy4 = dfmy3[(dfmy3["StartDate"] >= dfmy3["CalvingDate"]) & (dfmy3["StartDate"] <= dfmy3["upper_limit"])]

"""
SE_Number = ["SE-064c0cec-1189"]
dfmy4 = dfmy4[dfmy4["SE_Number"].isin(SE_Number)]
"""

# Make DIM
dfmy4 = dfmy4.copy()
dfmy4["StartDate"] = pd.to_datetime(dfmy4["StartDate"])
dfmy4["CalvingDate"] = pd.to_datetime(dfmy4["CalvingDate"])
dfmy4["DaysInMilk"] = (dfmy4["StartDate"] - dfmy4["CalvingDate"]).dt.days + 1
dfmy4.to_csv("MY.csv", index=False)

# ADD WEATHER DATA TO MY.CSV FILE
- uses weather data from AllPreProcessedWeatherData for each herd and MY.csv
- creates weather3.csv, and several MYn.csv to look at data with final dataset MY7.csv

In [ ]:
#
#
# PREPARATIONS FOR COMBINING MY AND WEATHER DATA
# Make next_calving
dfins6 = pd.read_csv("MY.csv", low_memory=False)
dfins7 = dfins6.drop_duplicates(subset=["SE_Number", "LactationNumber", "CalvingDate"])

col_keep = ["SE_Number", "CalvingDate", "LactationNumber"]
dfins7 = dfins7[col_keep]
dfins7["next_calving"] = dfins7.groupby(["SE_Number"])["CalvingDate"].shift(-1)
dfins7 = dfins6.merge(dfins7, on=["SE_Number", "LactationNumber", "CalvingDate"], how="left")

"""
Subset chosen cow
SE_Number = ["SE-064c0cec-1189"]
dfins7 = dfins7[dfins7["SE_Number"].isin(SE_Number)]
"""

# Add CullingDate to MY file
dfins8 = pd.read_csv("updateDF.csv", low_memory=False)
dfins8 = dfins8.drop_duplicates(subset=["SE_Number", "CullingDate"])
col_keep = ["SE_Number", "CullingDate"]
dfins8 = dfins8[col_keep]
dfins8 = dfins7.merge(dfins8, on=["SE_Number"], how="left")
dfins8.to_csv("MY3.csv", index=False)

col_keep = ["FarmName_Pseudo", "SE_Number", "LactationNumber", "CalvingDate", "next_calving", "upper_limit",
            "StartDate"]
df = dfins8[col_keep]

"""
SE_Number = ["SE-064c0cec-1189"]
df = df[df["SE_Number"].isin(SE_Number)]
"""

# keep only one obs per date
df2 = df.sort_values(by=["SE_Number", "LactationNumber", "StartDate"])
df2 = df2.drop_duplicates(subset=["SE_Number", "LactationNumber", "StartDate"])

df2 = df2.copy()
df2["CalvingDate"] = pd.to_datetime(df2["CalvingDate"])
df2["upper_limit"] = pd.to_datetime(df2["upper_limit"])

# Make a date variable for weather data - DateMeterological
# Create a list to hold new data
new_data = []

# Loop through each row in the original DataFrame
for _, row in df2.iterrows():
    herd = row["FarmName_Pseudo"]
    cow = row['SE_Number']
    lactation = row['LactationNumber']
    next_calving = row["next_calving"]
    start_date = row['CalvingDate']
    end_date = row['upper_limit']

    # Generate a date range
    date_range = pd.date_range(start=start_date, end=end_date)

    # Create a new DataFrame for Cow, Lactation, and date range
    temp_df = pd.DataFrame({
        "FarmName_Pseudo": herd,
        'SE_Number': cow,
        'LactationNumber': lactation,
        "next_calving": next_calving,
        'DateMeterological': date_range
    })

    # Append the new DataFrame to the list
    new_data.append(temp_df)

# Concatenate all the new DataFrames into one
df_exp = pd.concat(new_data, ignore_index=True)
df_exp.to_csv("MY4.csv", index=False)

# Remove duplicates
df_exp = df_exp.sort_values(by=["SE_Number", "LactationNumber", "DateMeterological"])
df_exp = df_exp.drop_duplicates(subset=["SE_Number", "LactationNumber", "next_calving", "DateMeterological"])

# only keep relevant data within calving interval
# Ensure both columns are in datetime format
df_exp["DateMeterological"] = pd.to_datetime(df_exp["DateMeterological"])
df_exp["next_calving"] = pd.to_datetime(df_exp["next_calving"])

# Set values in DateMeterological to NaN if they exceed the values in next_calving
df_exp.loc[df_exp["DateMeterological"] >= df_exp["next_calving"], "DateMeterological"] = np.nan
df_exp = df_exp.dropna(subset=['DateMeterological'])
col_keep = ["FarmName_Pseudo", "SE_Number", "LactationNumber", "DateMeterological"]
df_exp = df_exp[col_keep]
df_exp.to_csv("MY5.csv", index=False)

In [ ]:
#
#
# COMBINE WEATHER DATA FROM HERDS
# Load Global irradiance, THI_adj etc.
# set as comment after first time you run program, otherwise multiple columns in dataframe
# weather1-15a were "AllWeatherData", creating weather.csv and weather2.csv
# however, "AllPreProcessedWeatherData" contains full data needed, creating only weather3.csv
weather1b = pd.read_csv("AllPreProcessedWeatherData/processed_data_4eab8365.csv", delimiter=',', low_memory=False)
weather1b['FarmName_Pseudo'] = '4eab8365'
weather1b.to_csv("AllPreProcessedWeatherData/processed_data_4eab8365.csv", index=False)

weather2b = pd.read_csv("AllPreProcessedWeatherData/processed_data_5b581702.csv", delimiter=',', low_memory=False)
weather2b['FarmName_Pseudo'] = '5b581702'
weather2b.to_csv("AllPreProcessedWeatherData/processed_data_5b581702.csv", index=False)

weather3b = pd.read_csv("AllPreProcessedWeatherData/processed_data_5c06d92d.csv", delimiter=',', low_memory=False)
weather3b['FarmName_Pseudo'] = '5c06d92d'
weather3b.to_csv("AllPreProcessedWeatherData/processed_data_5c06d92d.csv", index=False)

weather4b = pd.read_csv("AllPreProcessedWeatherData/processed_data_5f7f33d6.csv", delimiter=',', low_memory=False)
weather4b['FarmName_Pseudo'] = '5f7f33d6'
weather4b.to_csv("AllPreProcessedWeatherData/processed_data_5f7f33d6.csv", index=False)

weather5b = pd.read_csv("AllPreProcessedWeatherData/processed_data_80b99061.csv", delimiter=',', low_memory=False)
weather5b['FarmName_Pseudo'] = '80b99061'
weather5b.to_csv("AllPreProcessedWeatherData/processed_data_80b99061.csv", index=False)

weather6b = pd.read_csv("AllPreProcessedWeatherData/processed_data_169e580a.csv", delimiter=',', low_memory=False)
weather6b['FarmName_Pseudo'] = '169e580a'
weather6b.to_csv("AllPreProcessedWeatherData/processed_data_169e580a.csv", index=False)

weather7b = pd.read_csv("AllPreProcessedWeatherData/processed_data_752efd72.csv", delimiter=',', low_memory=False)
weather7b['FarmName_Pseudo'] = '752efd72'
weather7b.to_csv("AllPreProcessedWeatherData/processed_data_752efd72.csv", index=False)

weather8b = pd.read_csv("AllPreProcessedWeatherData/processed_data_540275a1.csv", delimiter=',', low_memory=False)
weather8b['FarmName_Pseudo'] = '540275a1'
weather8b.to_csv("AllPreProcessedWeatherData/processed_data_540275a1.csv", index=False)

weather9b = pd.read_csv("AllPreProcessedWeatherData/processed_data_a624fb9a.csv", delimiter=',', low_memory=False)
weather9b['FarmName_Pseudo'] = 'a624fb9a'
weather9b.to_csv("AllPreProcessedWeatherData/processed_data_a624fb9a.csv", index=False)

weather10b = pd.read_csv("AllPreProcessedWeatherData/processed_data_a756bc39.csv", delimiter=',', low_memory=False)
weather10b['FarmName_Pseudo'] = 'a756bc39'
weather10b.to_csv("AllPreProcessedWeatherData/processed_data_a756bc39.csv", index=False)

weather11b = pd.read_csv("AllPreProcessedWeatherData/processed_data_ab18b151.csv", delimiter=',', low_memory=False)
weather11b['FarmName_Pseudo'] = 'ab18b151'
weather11b.to_csv("AllPreProcessedWeatherData/processed_data_ab18b151.csv", index=False)

weather12b = pd.read_csv("AllPreProcessedWeatherData/processed_data_ad0a39f5.csv", delimiter=',', low_memory=False)
weather12b['FarmName_Pseudo'] = 'ad0a39f5'
weather12b.to_csv("AllPreProcessedWeatherData/processed_data_ad0a39f5.csv", index=False)

weather13b = pd.read_csv("AllPreProcessedWeatherData/processed_data_afdd9a78.csv", delimiter=',', low_memory=False)
weather13b['FarmName_Pseudo'] = 'afdd9a78'
weather13b.to_csv("AllPreProcessedWeatherData/processed_data_afdd9a78.csv", index=False)

weather14b = pd.read_csv("AllPreProcessedWeatherData/processed_data_f454e660.csv", delimiter=',', low_memory=False)
weather14b['FarmName_Pseudo'] = 'f454e660'
weather14b.to_csv("AllPreProcessedWeatherData/processed_data_f454e660.csv", index=False)

weather15b = pd.read_csv("AllPreProcessedWeatherData/processed_data_fced84e9.csv", delimiter=',', low_memory=False)
weather15b['FarmName_Pseudo'] = 'fced84e9'
weather15b.to_csv("AllPreProcessedWeatherData/processed_data_fced84e9.csv", index=False)

# Get a list of all CSV files in the directory (adjust path as needed)
all_files = glob.glob("C:/Users/pagd0001/Desktop/Gigacow/Data/20240531/Gigacow_240531/AllPreProcessedWeatherData/*.csv")

# Initialize an empty list to store DataFrames
dfs = []

# Loop through the list of files and read them into DataFrames
for file in all_files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate all DataFrames into one
mdf2 = pd.concat(dfs, ignore_index=True)
mdf2.to_csv("weather3.csv", index=False)

In [ ]:
#
#
# COMBINE MY.csv and weather3.csv
# Add time stamps
# Weather file
daily = pd.read_csv("weather3.csv", low_memory=False)
daily.drop_duplicates(subset=["FarmName_Pseudo", "Tid"])

# Add time to MY file
MY = pd.read_csv("MY5.csv", low_memory=False)
MY.drop_duplicates(subset=["FarmName_Pseudo", "SE_Number", "LactationNumber", "DateMeterological"])
print(MY.shape)  # 1,127,307

daily['Tid'] = pd.to_datetime(daily['Tid'])
MY['DateMeterological'] = pd.to_datetime(MY['DateMeterological'])
MY.rename(columns={"DateMeterological": "StartDate"}, inplace=True)

MY3 = pd.read_csv("MY3.csv", low_memory=False)
col_keep = ["FarmName_Pseudo", "SE_Number", "LactationNumber", "StartDate", "StartTime"]
MY3 = MY3[col_keep]
MY3['StartDate'] = pd.to_datetime(MY3['StartDate'])
dfm = MY.merge(MY3, on=["FarmName_Pseudo", "SE_Number", "LactationNumber", "StartDate"], how="left")

# Step 1: Convert 'StartTime' column to pandas datetime, allowing NaT for missing values
dfm['StartTime'] = pd.to_datetime(dfm['StartTime'], format='%H:%M:%S', errors='coerce')

# Step 2: Round time to the nearest hour
dfm['RoundedTime'] = dfm['StartTime'].dt.round('h')

# Step 3: Fill missing values (NaT) with '00:00:00'
# Since 'RoundedTime' is a datetime column, use '00:00:00' as a time
dfm['RoundedTime'] = dfm['RoundedTime'].fillna(pd.Timestamp('00:00:00'))

# Step 4: Extract the hour from the RoundedTime column
dfm['Hour'] = dfm['RoundedTime'].dt.hour
# Extract the time part from the 'DateTime' column
dfm['StartTime'] = dfm['StartTime'].fillna('00:00:00')
dfm['TimeOnly'] = dfm['StartTime'].dt.time

# Save
col_keep = ["FarmName_Pseudo", "SE_Number", "LactationNumber", "StartDate", "TimeOnly", "Hour"]
dfm = dfm[col_keep]
dfm.rename(columns={"TimeOnly": "StartTime"}, inplace=True)
dfm.to_csv("MY5a.csv", index=False)

# Make Hour in weather dataset
daily['Hour'] = daily['Tid'].dt.hour
daily.to_csv("MY5b.csv", index=False)

daily['StartDate'] = pd.to_datetime(daily['StartDate'])
dfm['StartDate'] = pd.to_datetime(dfm['StartDate'])

# Merge MY with weather data FarmName, Date, and Hour
dfm = dfm.merge(daily, on=['FarmName_Pseudo', 'StartDate', 'Hour'], how='inner')
dfm = dfm.sort_values(by=["FarmName_Pseudo", "SE_Number", "LactationNumber", "StartDate", "StartTime"])
dfm = dfm.drop_duplicates(subset=["SE_Number", "LactationNumber", "StartDate", "StartTime"])

"""
SE_Number = ["SE-064c0cec-1189"]
dfm = dfm[dfm["SE_Number"].isin(SE_Number)]
"""

col_keep = ["FarmName_Pseudo", "SE_Number", "LactationNumber", "StartDate", "StartTime", "Hour",
            "Temperatur", "Relativ fuktighet", "Vindhastighet", "Vindriktning", "Byvind", "Nederbörd", "Snö",
            "Nederbördstyp", "Molnighet", "Sikt", "Lufttryck", "Global irradiance", "THI_adj", "HW", "cum_HW",
            "Temp15Threshold"]
dfm = dfm[col_keep]

dfm.rename(columns={"Temperatur": "Temperature", "Relativ fuktighet": "RelativeHumidity",
                    "Vindhastighet": "WindSpeed", "Vindriktning": "WindDirection",
                    "Byvind": "Crosswind", "Nederbörd": "Precipitation", "Snö": "Snow",
                    "Nederbördstyp": "PrecipitationType", "Molnighet": "Cloudiness",
                    "Sikt": "Visibility", "Lufttryck": "AirPressure",
                    "Global irradiance": "GlobalIrradiance"}, inplace=True)

dfm.to_csv("MY5c.csv", index=False)

missing_count = dfm['THI_adj'].isna().sum()
print(f"Number of missing observations in 'THI_adj': {missing_count}")  # 0
print(dfm.shape)  # 1,213,041

In [ ]:
#
#
# Add the rest of the MY data
MY2 = pd.read_csv("MY5c.csv", low_memory=False)

MY3 = pd.read_csv("MY3.csv", low_memory=False)
col_keep = ["FarmName_Pseudo", "SE_Number", "CalvingDate", "LactationNumber", "StartDate", "StartTime", "SessionNumber",
            "TotalYield", "DaysInMilk"]
MY3 = MY3[col_keep]

dfm2 = MY3.merge(MY2, on=["FarmName_Pseudo", "SE_Number", "LactationNumber", "StartDate", "StartTime"], how="left")
dfm2.to_csv("MY7.csv", index=False)

# ADD MILKING PARAMETERS FROM VMS HERDS
- adding Del_Milk_Robot to MY7.csv thus creating MY_weather.csv

In [ ]:
#
#
# ADD MILKING PARAMETERS FROM VMS
SCC = pd.read_csv("Del_Milk_Robot240918.csv", delimiter=';', low_memory=False)
print(f"No. of milking events in database: {SCC.shape}")  # 2,253,923obs
SCC = SCC.drop_duplicates(subset=["SE_Number", "MilkingStartDateTime"])
print(f"No. of unique milking events in database: {SCC.shape}")  # 2,244,467obs

""" 
# Check how many observations are missing timestamp
invalid_rows = SCC[pd.to_datetime(SCC['MilkingStartDateTime'], errors='coerce').isna()]
print(invalid_rows)
invalid_rows.to_csv("updateDF2.csv", index=False)  # 1574 obs
"""

SCC['MilkingStartDateTime'] = pd.to_datetime(SCC['MilkingStartDateTime'], errors='coerce')
# Identify rows where the time is missing or set to '00:00:00'
missing_time_rows = SCC[SCC['MilkingStartDateTime'].dt.time == pd.to_datetime('00:00:00').time()]

# Add a time of '00:00:00' to these rows (since they're already '00:00:00', no changes needed)
# Just for clarity, we reset the time to '00:00:00'
missing_time_rows['MilkingStartDateTime'] = missing_time_rows['MilkingStartDateTime'].apply(
    lambda x: x.replace(hour=0, minute=0, second=0)
)

# Merge the updated rows back onto the original DataFrame
# First, remove these rows from the original DataFrame
SCC_no_missing = SCC[SCC['MilkingStartDateTime'].dt.time != pd.to_datetime('00:00:00').time()]

# Concatenate the original DataFrame (without missing times) and the updated rows
SCC_updated = pd.concat([SCC_no_missing, missing_time_rows], ignore_index=True)
SCC_updated.to_csv("Updated_VMS.csv", index=False)

# Split into separate date and time columns
SCC_updated['MilkingStartDateTime'] = pd.to_datetime(SCC_updated['MilkingStartDateTime'])
SCC_updated['StartDate'] = SCC_updated['MilkingStartDateTime'].dt.date    # Extract the date part
SCC_updated['StartTime'] = SCC_updated['MilkingStartDateTime'].dt.time    # Extract the time part
SCC_updated['StartDate'] = pd.to_datetime(SCC_updated['StartDate'])
SCC_updated['StartTime'] = pd.to_datetime(SCC_updated['StartTime'], format='%H:%M:%S').dt.time

# Keep only MY data until 2024-08-18
cutoff_date = '2024-08-18'
SCC_updated = SCC_updated[SCC_updated['StartDate'] <= cutoff_date]
print(f"No. of unique milking events in database: {SCC_updated.shape}")  # 2,218,022

col_keep = ["FarmName_Pseudo", "SE_Number", "StartDate", "StartTime",
            "TotalYieldLF", "TotalYieldRF", "TotalYieldLR", "TotalYieldRR",
            "KickOffLF", "KickOffLR", "KickOffRF", "KickOffRR",
            "IncompleteLF", "IncompleteLR", "IncompleteRF", "IncompleteRR",
            "NotMilkedTeatLF", "NotMilkedTeatLR", "NotMilkedTeatRF", "NotMilkedTeatRR",
            "AverageFlowLF", "AverageFlowLR", "AverageFlowRF", "AverageFlowRR",
            "PeakFlowLF", "PeakFlowLR", "PeakFlowRF", "PeakFlowRR",
            "BloodLF", "BloodLR", "BloodRF", "BloodRR",
            "ConductivityLF", "ConductivityLR", "ConductivityRF", "ConductivityRR",
            "Occ"]
SCC_updated = SCC_updated[col_keep]

df2 = pd.read_csv("MY7.csv", low_memory=False)
print(f"No. of unique milking events in MY7: {df2.shape}")  # 2,299,007

df2['StartDate'] = pd.to_datetime(df2['StartDate'])
df2['StartTime'] = pd.to_datetime(df2['StartTime'], format='%H:%M:%S').dt.time
SCC2 = df2.merge(SCC_updated, on=["FarmName_Pseudo", "SE_Number", "StartDate", "StartTime"], how="left")  # changed from outer

"""
# How dos this data look?
frequency_table = SCC2['KickOffLF'].value_counts()
print(frequency_table)
frequency_table = SCC2['IncompleteLF'].value_counts()
print(frequency_table)
frequency_table = SCC2['NotMilkedTeatLF'].value_counts()
print(frequency_table)
"""

# List of columns where you want to replace commas with dots
columns_to_modify = ["TotalYieldLF", "TotalYieldRF", "TotalYieldLR", "TotalYieldRR",
                     "AverageFlowLF", "AverageFlowLR", "AverageFlowRF", "AverageFlowRR",
                     "PeakFlowLF", "PeakFlowLR", "PeakFlowRF", "PeakFlowRR",
                     "BloodLF", "BloodLR", "BloodRF", "BloodRR",
                     "ConductivityLF", "ConductivityLR", "ConductivityRF", "ConductivityRR",
                     "Occ"]

# Replace comma with dot and convert to float
SCC2[columns_to_modify] = SCC2[columns_to_modify].replace(',', '.', regex=True).astype(float)
SCC2.to_csv("MY8.csv", index=False)

# Change order of columns
new_column_order = ["FarmName_Pseudo", "SE_Number", "CalvingDate", "LactationNumber",
                    "DaysInMilk", "StartDate", "StartTime", "SessionNumber", "TotalYield",
                    "TotalYieldLF", "TotalYieldRF", "TotalYieldLR", "TotalYieldRR",
                    "AverageFlowLF", "AverageFlowLR", "AverageFlowRF", "AverageFlowRR",
                    "PeakFlowLF", "PeakFlowLR", "PeakFlowRF", "PeakFlowRR",
                    "BloodLF", "BloodLR", "BloodRF", "BloodRR",
                    "ConductivityLF", "ConductivityLR", "ConductivityRF", "ConductivityRR",
                    "Occ",
                    "Temperature", "RelativeHumidity", "WindSpeed", "WindDirection", "Crosswind", "Precipitation",
                    "Snow", "PrecipitationType", "Cloudiness", "Visibility", "AirPressure", "GlobalIrradiance",
                    "THI_adj", "HW", "cum_HW", "Temp15Threshold"]
SCC2 = SCC2[new_column_order]
SCC2.to_csv("MY8.csv", index=False)

# Add final ID-numbers from DelPro
df = pd.read_csv("Del_CowMilkYield_Common240823.csv", delimiter=';', low_memory=False)
col_keep = ["SE_Number", "AnimalNumber", "Del_Cow_Id"]
df = df[col_keep]
df = df.drop_duplicates(subset=["SE_Number", "AnimalNumber", "Del_Cow_Id"])

my = pd.read_csv("MY8.csv", low_memory=False)
# my = my.drop(columns=["AnimalNumber", "Del_Cow_Id"])
mdf = pd.merge(my, df, on=["SE_Number"], how="left")
new_column_order = ["FarmName_Pseudo", "SE_Number", "AnimalNumber", "Del_Cow_Id", "CalvingDate", "LactationNumber",
                    "DaysInMilk", "StartDate", "StartTime", "SessionNumber", "TotalYield",
                    "TotalYieldLF", "TotalYieldRF", "TotalYieldLR", "TotalYieldRR",
                    "AverageFlowLF", "AverageFlowLR", "AverageFlowRF", "AverageFlowRR",
                    "PeakFlowLF", "PeakFlowLR", "PeakFlowRF", "PeakFlowRR",
                    "BloodLF", "BloodLR", "BloodRF", "BloodRR",
                    "ConductivityLF", "ConductivityLR", "ConductivityRF", "ConductivityRR",
                    "Occ",
                    "Temperature", "RelativeHumidity", "WindSpeed", "WindDirection", "Crosswind", "Precipitation",
                    "Snow", "PrecipitationType", "Cloudiness", "Visibility", "AirPressure", "GlobalIrradiance",
                    "THI_adj", "HW", "cum_HW", "Temp15Threshold"]
mdf = mdf[new_column_order]
mdf.to_csv("MY_weather.csv", index=False)

# MISCELLANEOUS MINOR EDITS
- Add full ID-numbers from DelPro
- Retransfer breed
- Double check for duplicates in MY 

Creates the following datasets:
- updateDF.csv: containing data for cows for future use in HeatStress project
- MY_weather.csv: containing MY and weather data for future use in HeatStress project
- dfForSreten.csv: containing data for cows for delivery to Sreten Andonov
- MY_weatherForSreten.csv: containing MY and weather data for delivery to Sreten Andonov

In [ ]:
#
#
# Add final ID-numbers from DelPro
df = pd.read_csv("Del_CowMilkYield_Common240823.csv", delimiter=';', low_memory=False)
col_keep = ["SE_Number", "AnimalNumber", "Del_Cow_Id"]
df = df[col_keep]
df = df.drop_duplicates(subset=["SE_Number", "AnimalNumber", "Del_Cow_Id"])

my = pd.read_csv("updateDF.csv", low_memory=False)
mdf = pd.merge(my, df, on=["SE_Number"], how="left")
new_column_order = ["FarmName_Pseudo", "SE_Number", "AnimalNumber", "Del_Cow_Id", "Breed", "BirthDate",
                    "Father_SE_Number", "Mother_SE_Number", "CalvingDate", "LactationNumber",
                    "InseminationDate", "PregnancyCheckDate", "PregnancyStatus", "DryOffDate", "CullingDate",
                    "ExitReason_PrimaryReasonKok", "ExitReason_SecondaryReason1Kok", "ExitReason_SecondaryReason2Kok",
                    "CullingReason1", "CullingReason2"]
mdf = mdf[new_column_order]
mdf.to_csv("updateDF.csv", index=False)

In [ ]:
#
#
# TRANSFER BREED CORRECTLY TO updateDF.csv
df = pd.read_csv("breed.csv", low_memory=False)
df2 = df.drop_duplicates(subset=["SE_Number", "Breed"])
print(df2.shape)  # 9,535
df2 = df.drop_duplicates(subset=["SE_Number"])
print(df2.shape)  # 9,535
df.drop(columns=["FarmName_Pseudo", "SireBreedKok", "DamBreedKok", "MGSBreedKok", "BreedNameDelPro", "BreedKok", "BreedDelPro"], inplace=True)

df3 = pd.read_csv("updateDF.csv", low_memory=False)
df3.drop(columns=["Breed"], inplace=True)

df4 = df.merge(df3, on="SE_Number", how="left")
df4.to_csv("updateDF.csv", index=False)
df4.to_csv("dfForSreten.csv", index=False)

#
#
# TRANSFER BREED TO MY_weather.csv
df = pd.read_csv("breed.csv", low_memory=False)
df2 = df.drop_duplicates(subset=["SE_Number", "Breed"])
print(df2.shape)  # 9,535
df2 = df.drop_duplicates(subset=["SE_Number"])
print(df2.shape)  # 9,535
df.drop(columns=["SireBreedKok", "DamBreedKok", "MGSBreedKok", "BreedNameDelPro", "BreedKok", "BreedDelPro"], inplace=True)

df3 = pd.read_csv("MY_weather.csv", low_memory=False)

df4 = df.merge(df3, on=["FarmName_Pseudo", "SE_Number"], how="left")
df4 = df4[df4['LactationNumber'].notna()]

"""
SE_Number = ["SE-064c0cec-1189"]
df4 = df4[df4["SE_Number"].isin(SE_Number)]
"""
df4.to_csv("MY_weather.csv", index=False)

In [ ]:
# DOUBLE CHECK FOR DUPLICATES IN MY_WEATHER FILE FROM MERGE DUE TO DATA EXTRACTION???
df = pd.read_csv("MY_weather.csv", low_memory=False)
print(f"No. of records in crude data: {df.shape}")  # 2,307,309
df3a = df.drop_duplicates(subset=['SE_Number', "LactationNumber", "StartDate", "StartTime", "SessionNumber", "DaysInMilk"])
print(f"No. of records in MY_weather.csv file: {df3a.shape}")  # 1,868,019
df3a.to_csv("MY_weather.csv", index=False)
df3a.to_csv("MY_weatherForSreten.csv", index=False)

In [ ]:
#
#
# Descriptive statistics for milking file
df = pd.read_csv("MY_weather.csv", low_memory=False)
print(f"No. of unique milking events in database: {df.shape}")  # 2,299,007 milking events
df2 = df.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. of lactations in milking file: {df2.shape}")  # 3,050 lactations
df2 = df.drop_duplicates(subset=["SE_Number"])
print(f"No. of cows in milking file: {df2.shape}")  # 1,659 cows
df2 = df.drop_duplicates(subset=["FarmName_Pseudo"])
print(f"No. of herds in milking file: {df2.shape}")  # 7 herds

col_keep = ["FarmName_Pseudo"]
df2 = df2[col_keep]
print(f"Herds in unfiltered data: \n", df2.to_string(index=False))

In [ ]:
#
#
# SAVE IN updateDF.csv ONLY DATA FROM COWS AND LACTATIONS WITH MILK YIELD DATA
df = pd.read_csv("updateDF.csv", low_memory=False)

dfa = df.drop_duplicates(subset=['SE_Number', "LactationNumber", "InseminationDate", "PregnancyCheckDate"])
print(f"No. of pregnancy checks in updateDF file: {dfa.shape}")  # 47,326
dfa = df.drop_duplicates(subset=['SE_Number', "LactationNumber", "InseminationDate"])
print(f"No. of inseminations in updateDF file: {dfa.shape}")  # 36,506
dfa = df.drop_duplicates(subset=['SE_Number', "LactationNumber"])
print(f"No. of lactations in updateDF file: {dfa.shape}")  # 20,683
dfa = df.drop_duplicates(subset=['SE_Number'])
print(f"No. of cows in updateDF: {dfa.shape}")  # 9,535

df2 = pd.read_csv("MY_weather.csv", low_memory=False)
df2["tag"] = 1
col_keep = ["SE_Number", "LactationNumber", "tag"]
df2 = df2[col_keep]
df2 = df2.drop_duplicates(subset=['SE_Number', "LactationNumber"])
print(df2.shape)  # 3,050
df2.to_csv("updateDF2.csv", index=False)

df3 = df2.merge(df, on=["SE_Number", "LactationNumber"], how="left")
df3 = df3[df3['tag'] == 1]
df3 = df3.drop(columns=["tag"])
"""
SE_Number = ["SE-064c0cec-1189"]
df3 = df3[df3["SE_Number"].isin(SE_Number)]
"""
df3.to_csv("updateDF.csv", index=False)
df3.to_csv("dfForSreten.csv", index=False)

df3a = df3.drop_duplicates(subset=['SE_Number', "LactationNumber", "InseminationDate", "PregnancyCheckDate"])
print(f"No. of pregnancy checks in updateDF file: {df3a.shape}")  # 8,336
df3a = df3.drop_duplicates(subset=['SE_Number', "LactationNumber", "InseminationDate"])
print(f"No. of inseminations in updateDF file: {df3a.shape}")  # 5,592
df3a = df3.drop_duplicates(subset=['SE_Number', "LactationNumber"])
print(f"No. of lactations in updateDF file: {df3a.shape}")  # 3,050
df3a = df3.drop_duplicates(subset=['SE_Number'])
print(f"No. of cows in updateDF: {df3a.shape}")  # 1,659

# FINAL COMMENTS:
- MY and weather data can be found in MY_weather.csv
- Basic data for each cow is in updateDF.csv
- Structure and codes corresponding to cow database whenever possible,
    see "Produktbeskrivning" and "Kokförteckning" from Växa
- First data from milking parlours, then from VMSs followed by weather data
- MESAN data from SMHI are on hourly basis, and has been matched to the closes hour of the milking event
        e.g. if the cow was milked 6:20, corresponding weather data is from 6:00
        e.g. if the cow was milked 16:45, corresponding weather data is from 17:00
- MESAN data range covers 2022-01-01 to 2024-08-18 for all herds
- Range of MY data from Gigacow herds depend on when the herd was connected
- For basic production data, primary source is the cow database on the assumption that this information is superior
    to the herd management system. In the event of missing data from the cow database, data from the management system
    is added
    Note: A lot of cows will lack AnimalNumber, Del_Cow_Id because of primary source is from the cow database
- Complementary data from cow database is updated once every quarter 
- 'CullingReason1' and 'CullingReason2' are the combined culling reasons as recorded in herd management system and cow 
    database, however, kept originals due to free text recording can vary
- PregnancyStatus was originally free text alternatives but has been changed to coincide with cow database codes,
    see "Kokförteckning" from Växa

In [ ]:
#
#
# PLOTTING TOTAL DAILY MILK YIELD BY HERD IN CRUDE DATA
# Load data
df_lact = pd.read_csv("MY_weather.csv", low_memory=False)

# Ensure the StartDate column is a datetime object
df_lact['StartDate'] = pd.to_datetime(df_lact['StartDate'])

# Keep only MY data from 2022-01-01 until 2024-08-18
cutoff_date1 = '2022-01-01'
cutoff_date2 = '2024-08-18'
df_lact = df_lact[df_lact['StartDate'] >= cutoff_date1]
df_lact = df_lact[df_lact['StartDate'] <= cutoff_date2]

# list of farms
list_of_farms = list(df_lact["FarmName_Pseudo"].unique())

# Dictionary to store farm color mapping
farm_color_mapping = {}

# Loop through each farm and create a separate plot for each
for i, farm in enumerate(list_of_farms):
    # Create a new figure for each farm
    plt.figure(figsize=(12, 6))

    selected_farm = df_lact[df_lact['FarmName_Pseudo'] == farm]
    number_of_cows = len(selected_farm['SE_Number'].unique())
    daily_yield = selected_farm.groupby('StartDate')['TotalYield'].sum()

    # Get color for this farm, ensuring enough colors
    color = sns.color_palette('bright', n_colors=max(len(list_of_farms), 10))[i]
    farm_color_mapping[farm] = color

    # Generate a complete date range and reindex the daily yield
    all_dates = pd.date_range(start=daily_yield.index.min(), end=daily_yield.index.max(), freq='D')
    daily_yield = daily_yield.reindex(all_dates, fill_value=0)

    # Plot data
    plt.plot(daily_yield.index, daily_yield.values, label=f'Farm {farm} \n{number_of_cows} cows', color=color)
    plt.title(f'Milk Data for Farm {farm}', fontsize=16)
    plt.ylabel('Daily Yield', fontsize=14)
    plt.legend(loc='upper right')
    plt.grid(True)

    # Set x-axis label
    plt.xlabel('Date', fontsize=14)

    # Show the plot
    plt.tight_layout()
    plt.show()


In [ ]:
#
#
# Number of Instances of TotalYield for Each Farm on Each Day
data = pd.read_csv("MY_weather.csv", low_memory=False)

# Group the data by 'FarmName_Pseudo' and 'StartDate', and get the count of 'TotalYield'
count_data = data.groupby(['FarmName_Pseudo', 'StartDate'])['TotalYield'].count().reset_index(name='Count')

# Convert 'StartDate' to datetime
count_data['StartDate'] = pd.to_datetime(count_data['StartDate'])

# Sort the data by date to ensure the labels match the ticks
count_data.sort_values('StartDate', inplace=True)

# Pivot to have 'StartDate' as columns and 'FarmName_Pseudo' as index
pivot_data = count_data.pivot(index='FarmName_Pseudo', columns='StartDate', values='Count').fillna(0)

# Plot data
plt.figure(figsize=(12, 8))
ax = sns.heatmap(pivot_data, cmap='Purples', cbar_kws={'label': 'Count'})

num_days = len(pivot_data.columns)
days_between_ticks = num_days // 6  # Adjust this as needed

ax.set_xticks(range(0, num_days, days_between_ticks))
ax.set_xticklabels([date.strftime('%Y-%m-%d') for date in pivot_data.columns[::days_between_ticks]], rotation=45, ha='right')
plt.xlabel('Date', fontsize=14)
plt.ylabel('FarmName_Pseudo', fontsize=14)
plt.title('Number of Instances of TotalYield for Each Farm on Each Day', fontsize=16)

plt.tight_layout()
plt.show()

# FILTERING MILK YIELD DATA
- keep only NRDC, SH, SJB and dairy crosses
- keep only data between 2022-01-01 -- 2024-08-18
- have to start milking by 1-30DIM and maintain milking until 100-400DIM
- keep only 1-7 lactation (make parity 1, 2, +3)
- set MY between 2.5-60kg to handle outliers, kick-offs and incomplete milkings
- generates filtered dataset MY_weather2.csv


In [ ]:
#
#
# FILTER MILK YIELD DATA
df3 = pd.read_csv("MY_weather.csv", low_memory=False)
print(f"No. milking events in MY_weather.csv: {df3.shape}")  # 1,868,019

value_counts = df3['Breed'].value_counts()
print(value_counts)

# Pie chart over distribution of Breed
value_counts = df3['Breed'].value_counts()
plt.figure(figsize=(10, 10))
plt.pie(value_counts, labels=value_counts.index, autopct='%1.1f%%', startangle=140, pctdistance=0.85)
plt.axis('equal')
plt.title('Distribution of Milking Events Across Breeds in MY_weather.csv')
plt.show()

# Keep only SRB, SH, dairy crosses and SJB
df3 = df3[(df3["Breed"] == "NRDC") |
          (df3["Breed"] == "SLB") |
          (df3["Breed"] == "DairyCross") |
          (df3["Breed"] == "SJB")]
print(f"No. milking events in SRB, SH, dairy Crosses and SJB cows: {df3.shape}")  # 1,825,106

# No. lactations and cows in data
test = df3.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. of lactations from SRB, SH and SJB: {test.shape}")  # 2,938
test = test.drop_duplicates(subset=["SE_Number"])
print(f"No. of cows from SRB, SH and SJB: {test.shape}")  # 1,585

# Keep only data where have both MY and weather information
df3a = df3.drop_duplicates(subset=['SE_Number', "LactationNumber", "StartDate", "StartTime"])
print(f"No. of milking events in MY_weather.csv: {df3a.shape}")  # 1,414,770
df3a = df3.drop_duplicates(subset=['SE_Number', "LactationNumber", "StartDate"])
print(f"No. of milking days in MY_weather.csv: {df3a.shape}")  # 633,247
df3a = df3.drop_duplicates(subset=['SE_Number', "LactationNumber"])
print(f"No. of lactations in MY_weather.csv: {df3a.shape}")  # 2,938
df3a = df3.drop_duplicates(subset=['SE_Number'])
print(f"No. of cows in MY_weather.csv: {df3a.shape}")  # 1,585

# Keep only MY data from 2022-01-01 until 2024-08-18
cutoff_date1 = '2022-01-01'
cutoff_date2 = '2024-08-18'
df3 = df3[df3['StartDate'] >= cutoff_date1]
df3 = df3[df3['StartDate'] <= cutoff_date2]

df3a = df3.drop_duplicates(subset=['SE_Number', "LactationNumber", "StartDate", "StartTime"])
print(f"No. of milking events in MY_weather.csv: {df3a.shape}")  # 955,139 milking events
df3a = df3.drop_duplicates(subset=['SE_Number', "LactationNumber", "StartDate"])
print(f"No. of milking days in MY_weather.csv: {df3a.shape}")  # 469,384 milking days
df3a = df3.drop_duplicates(subset=['SE_Number', "LactationNumber"])
print(f"No. of lactations in MY_weather.csv: {df3a.shape}")  # 2,305 lactations
df3a = df3.drop_duplicates(subset=['SE_Number'])
print(f"No. of cows in MY_weather.csv: {df3a.shape}")  # 1,392 cows

non_missing_count = df3['THI_adj'].notna().sum()
missing_count = df3['THI_adj'].isna().sum()

print(f"Observations with data: {non_missing_count}")  # 744,624 obs
print(f"Observations with missing data: {missing_count}")  # 601,490 obs

df3.dropna(subset=['THI_adj'], inplace=True)
df3a = df3.drop_duplicates(subset=['SE_Number', "LactationNumber", "StartDate", "StartTime"])
print(f"No. of milking events in updateDF file: {df3a.shape}")  # 744,624 milking events
df3a = df3.drop_duplicates(subset=['SE_Number', "LactationNumber", "StartDate"])
print(f"No. of milking days in updateDF file: {df3a.shape}")  # 261,709 milking days
df3a = df3.drop_duplicates(subset=['SE_Number', "LactationNumber"])
print(f"No. of lactations in updateDF file: {df3a.shape}")  # 1,931 lactations
df3a = df3.drop_duplicates(subset=['SE_Number'])
print(f"No. of cows in updateDF: {df3a.shape}")  # 1,133 cows

# Filter to demand started milking by 1-30 DIM and maintain milking until 100-400 DIM
first_last_df = df3.groupby(['SE_Number', 'LactationNumber'])['DaysInMilk'].agg(['first', 'last']).reset_index()

plt.figure()
plt.hist(first_last_df["first"], color='blue')
plt.title(f"First DIM in Lactation")
plt.xlabel('First DIM')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

plt.figure()
plt.hist(first_last_df["last"], color='blue')
plt.title(f"Last DIM in Lactation")
plt.xlabel('Last DIM')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

df3 = df3.merge(first_last_df, on=["SE_Number", "LactationNumber"], how="left")
df3.to_csv("MY3.csv", index=False)

average_production = (df3.groupby(['FarmName_Pseudo'])['first']
                      .agg(['mean', 'std']).reset_index())
average_production['mean'] = average_production['mean'].round(0)
average_production['std'] = average_production['std'].round(0)
average_production.rename(columns={'mean': 'MeanFirstDIM', 'std': 'SDFirstDIM'}, inplace=True)
print(f"Mean and SD First DIM: \n", average_production.to_string(index=False))

average_production = (df3.groupby(['FarmName_Pseudo'])['last']
                      .agg(['mean', 'std']).reset_index())
average_production['mean'] = average_production['mean'].round(0)
average_production['std'] = average_production['std'].round(0)
average_production.rename(columns={'mean': 'MeanLastDIM', 'std': 'SDLastDIM'}, inplace=True)
print(f"Mean and SD Last DIM: \n", average_production.to_string(index=False))


def filter_first_last(group):
    first_value = group['DaysInMilk'].iloc[0]
    last_value = group['DaysInMilk'].iloc[-1]
    return (1 <= first_value <= 30) and (100 <= last_value <= 400)


filtered_df = df3.groupby(["SE_Number", "LactationNumber"]).filter(filter_first_last)
print(f"No. milking events in SRB, SH, SJB and dairy crosses cows within 1-30 DIM and 100-400 DIM: {filtered_df.shape}")
# 30DIM: 518,107

# No. lactations and cows in data
test = filtered_df.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. of lactations from SRB, SH, SJB and dairy crosses within 1-30DIM and 100-400DIM: {test.shape}")
# 30DIM: 1,173
test = filtered_df.drop_duplicates(subset=["SE_Number"])
print(f"No. of cows from SRB, SH, SJB and dairy crosses within 1-30DIM and 100-400DIM: {test.shape}")
# 30DIM: 863

# Distribution of milk yield data over various lactations
value_counts = filtered_df['LactationNumber'].value_counts()
print(value_counts)

# Keep only lactation 1-7
df_lact = filtered_df[filtered_df["LactationNumber"] <= 7]
print(f"No. milking events in SRB, SH, SJB and dairy crosses cows within 1-30 DIM and 100-400 DIM in lactation 1-7: "
      f"{df_lact.shape}")  # 30DIM: 517,235

# No. lactations and cows in data
test = df_lact.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. of lactations from SRB, SH, SJB and dairy crosses within 1-30DIM and 100-400DIM in lactation 1-7: "
      f"{test.shape}")  # 30DIM: 1,168
test = df_lact.drop_duplicates(subset=["SE_Number"])
print(f"No. of cows from SRB, SH, SJB and dairy crosses within 1-30DIM and 100-400DIM in lactation 1-7: {test.shape}")
# 30DIM: 861

# Make Parity 1-3
df_lact = df_lact.copy()
df_lact["Parity"] = df_lact["LactationNumber"]
df_lact.loc[(df_lact['LactationNumber'] >= 3) & (df_lact['LactationNumber'] <= 7), 'Parity'] = 3

# CHECK DISTRIBUTION OF TOTAL YIELD COLUMN
# Basic statistics
summary_stats = df_lact['TotalYield'].describe()
percentiles = np.percentile(df_lact['TotalYield'], [1, 5, 10, 90, 95, 99])

print("Descriptive Statistics:\n", summary_stats)
print("\nPercentiles (1%, 5%, 10%, 90%, 95%, 99%):", percentiles)

# count_my_rec = df.drop_duplicates(subset=["SE_Number", "LactationNumber"])
sns.boxplot(x='FarmName_Pseudo', y='TotalYield', data=df_lact)
plt.title('Box Plot of Total Yield per Milking Event Grouped by Herd')
plt.xlabel('Herd')
plt.ylabel('Total Yield per Milking Event')
plt.xticks(rotation=45)
plt.show()

# Count instances with TotalYield greater than 60 L
count_invalid_total_yield = df_lact[df_lact['TotalYield'] > 60].shape[0]
print(f"Number of instances with TotalYield greater than 60: {count_invalid_total_yield}")
# 156 observations - have been handled already, code below technically redundant
df_lact2 = df_lact[df_lact['TotalYield'] < 60]

count_invalid_total_yield = df_lact2[df_lact2['TotalYield'] < 2.5].shape[0]
print(f"Number of instances with TotalYield less than 2.5 kg: {count_invalid_total_yield}")  # 4839 observations
df_lact2 = df_lact2[df_lact2['TotalYield'] > 2.5]

df3a = df_lact2.drop_duplicates(subset=['SE_Number', "LactationNumber", "StartDate", "StartTime"])
print(f"No. of pregnancy checks in updateDF file: {df3a.shape}")  # 512,388 milking events
df3a = df_lact2.drop_duplicates(subset=['SE_Number', "LactationNumber", "StartDate"])
print(f"No. of inseminations in updateDF file: {df3a.shape}")  # 179,683 milking days
df3a = df_lact2.drop_duplicates(subset=['SE_Number', "LactationNumber"])
print(f"No. of lactations in updateDF file: {df3a.shape}")  # 1,168 lactations
df3a = df_lact2.drop_duplicates(subset=['SE_Number'])
print(f"No. of cows in updateDF: {df3a.shape}")  # 861 cows

# Basic statistics
summary_stats = df_lact2['TotalYield'].describe()
percentiles = np.percentile(df_lact2['TotalYield'], [1, 5, 10, 90, 95, 99])

print("Descriptive Statistics:\n", summary_stats)
print("\nPercentiles (1%, 5%, 10%, 90%, 95%, 99%):", percentiles)

df_lact2.to_csv("MY_weather2.csv", index=False)

In [ ]:
#
#
# DESCRIPTIVE STATISTICS FILTERED DATA
# Milking records
df_lact = pd.read_csv("MY_weather2.csv", low_memory=False)

count_my_rec = df_lact.groupby(["Parity"])["StartDate"].count().reset_index()
print(f"No. of milking records divided over parities: \n", count_my_rec.to_string(index=False))

count_my_rec = df_lact.groupby(["Parity", "Breed"])["StartDate"].count().reset_index()
print(f"No. of milking records divided over parities and breeds: \n", count_my_rec.to_string(index=False))

# By parity
for_my_rec5 = df_lact.drop_duplicates(subset=["SE_Number", "LactationNumber"])
print(f"No. of parities in milking file: {for_my_rec5.shape}")  # 1,168

count_my_rec = for_my_rec5.groupby(["Parity", "Breed"])["SE_Number"].count().reset_index()
print(f"No. of parities from SRB, SH, SJB and dairy crosses: \n", count_my_rec.to_string(index=False))

# By cows
for_my_rec4 = df_lact.drop_duplicates(subset=["SE_Number"])
print(f"No. of cows in milking file: {for_my_rec4.shape}")  # 861

for_my_rec5 = for_my_rec4.drop_duplicates(subset=["SE_Number"])
count_my_rec = for_my_rec5.groupby(["Breed"])["SE_Number"].count().reset_index()
print(f"No. of cows from SRB, SH, SJB and dairy crosses: \n", count_my_rec.to_string(index=False))

# Herd info
df_lact = pd.read_csv("MY_weather2.csv", low_memory=False)
df_lact = df_lact.drop_duplicates(subset=["FarmName_Pseudo"])
col_keep = ["FarmName_Pseudo"]
df_lact = df_lact[col_keep]
print(df_lact.shape)  # 5 herds
print(f"Herds in filtered data: \n", df_lact.to_string(index=False))

In [ ]:
#
#
# LOOK AT MILK YIELD BY HERD EACH DAY
# Load data
df_lact = pd.read_csv("MY_weather2.csv", low_memory=False)

# Ensure the StartDate column is a datetime object
df_lact['StartDate'] = pd.to_datetime(df_lact['StartDate'])

# list of farms
list_of_farms = list(df_lact["FarmName_Pseudo"].unique())

# Create a subplot for each farm
fig, axs = plt.subplots(len(list_of_farms), 1, figsize=(12, len(list_of_farms) * 4), sharex=True)

# If there's only one subplot, convert axs to a list
if len(list_of_farms) == 1:
    axs = [axs]

# Dictionary to store farm color mapping
farm_color_mapping = {}

# Loop through each farm and plot in its subplot
for i, farm in enumerate(list_of_farms):
    ax = axs[i]

    selected_farm = df_lact[df_lact['FarmName_Pseudo'] == farm]
    number_of_cows = len(selected_farm['SE_Number'].unique())
    daily_yield = selected_farm.groupby('StartDate')['TotalYield'].sum()

    # Get color for this farm, ensuring enough colors
    color = sns.color_palette('bright', n_colors=max(len(list_of_farms), 10))[i]
    farm_color_mapping[farm] = color

    # Generate a complete date range and reindex the daily yield
    all_dates = pd.date_range(start=daily_yield.index.min(), end=daily_yield.index.max(), freq='D')
    daily_yield = daily_yield.reindex(all_dates, fill_value=0)

    # Plot data
    ax.plot(daily_yield.index, daily_yield.values, label=f'Farm {farm} \n{number_of_cows} cows', color=color)
    ax.set_title(f'Milk Data for Farm {farm}', fontsize=16)
    ax.set_ylabel('Daily Yield', fontsize=14)
    ax.legend(loc='upper right')
    ax.grid(True)

# Set x-axis label for the last subplot
axs[-1].set_xlabel('Date', fontsize=14)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
#
#
# LIKE ABOVE BUT NOT SUBPLOTS
# Load data
df_lact = pd.read_csv("MY_weather2.csv", low_memory=False)

# Ensure the StartDate column is a datetime object
df_lact['StartDate'] = pd.to_datetime(df_lact['StartDate'])

# list of farms
list_of_farms = list(df_lact["FarmName_Pseudo"].unique())

# Dictionary to store farm color mapping
farm_color_mapping = {}

# Loop through each farm and create a separate plot for each
for i, farm in enumerate(list_of_farms):
    # Create a new figure for each farm
    plt.figure(figsize=(12, 6))

    selected_farm = df_lact[df_lact['FarmName_Pseudo'] == farm]
    number_of_cows = len(selected_farm['SE_Number'].unique())
    daily_yield = selected_farm.groupby('StartDate')['TotalYield'].sum()

    # Get color for this farm, ensuring enough colors
    color = sns.color_palette('bright', n_colors=max(len(list_of_farms), 10))[i]
    farm_color_mapping[farm] = color

    # Generate a complete date range and reindex the daily yield
    all_dates = pd.date_range(start=daily_yield.index.min(), end=daily_yield.index.max(), freq='D')
    daily_yield = daily_yield.reindex(all_dates, fill_value=0)

    # Plot data
    plt.plot(daily_yield.index, daily_yield.values, label=f'Farm {farm} \n{number_of_cows} cows', color=color)
    plt.title(f'Milk Data for Farm {farm}', fontsize=16)
    plt.ylabel('Daily Yield', fontsize=14)
    plt.legend(loc='upper right')
    plt.grid(True)

    # Set x-axis label
    plt.xlabel('Date', fontsize=14)

    # Show the plot
    plt.tight_layout()
    plt.show()